# Análise dos dados das variáveis relacionadas ao IDEB

O notebook contém experimentos com essas variáveis. São utilizadas duas bases de dados:<br>
- Váriáveis derivadas do SAEB (informações socioeconômicas)<br> 
- Variáveis com informações das escolas e professores<br>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, RFE, RFECV, SelectFromModel
from sklearn.feature_selection import mutual_info_classif, f_classif, chi2
from sklearn.preprocessing import LabelEncoder
import numpy as np
from numpy import array 
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

## Importando bases de dados

In [ ]:
#Importando variáveis derivadas do SAEB

#base_estudantes_se = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/variaveis_derivadas_se.csv')

#base_estudantes_se = pd.read_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_se.csv')

#base_estudantes_se.set_index('ID_ESCOLA', inplace=True)

#base_estudantes_al = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/variaveis_derivadas_al.csv')

#base_estudantes_al = pd.read_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_al.csv')

#base_estudantes_al.set_index('ID_ESCOLA', inplace=True)

base_estudantes_ba = pd.read_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_ba.csv')

dados_pessoais = base_estudantes_ba

#base_estudantes_ba.set_index('ID_ESCOLA', inplace=True)

#dados_pessoais = pd.merge(base_estudantes_se, base_estudantes_al, how = 'outer')

#dados_pessoais = pd.merge(base_estudantes_ba, dados_pessoais, how = 'outer')

In [ ]:
dados_pessoais.set_index('ID_ESCOLA', inplace=True)

In [ ]:
dados_pessoais

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_SITUACAO_CENSO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,...,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_B,TX_RESP_Q016_C,TX_RESP_Q016_D,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_C,TX_RESP_Q019_D,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_C,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_C,TX_RESP_Q004_D,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
ID_ESCOLA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
29211590,2017,2,29,2900108,2,2,1,1,36,36,65,7,58,14,63,6,47,20,68,0,65,3,62,6,66,1,49,19,22,44,21,43,44,23,46,21,63,7,2,49,...,6,1,8,16,29,17,1,6,23,14,5,13,4,7,8,21,10,8,8,2,10,0,8,25,15,8,4,1,8,9,5,10,3,7,5,5,3,6,6,7
29032539,2017,2,29,2900207,2,2,1,1,52,95,124,23,107,39,95,53,72,73,145,2,137,9,133,13,142,4,110,36,35,107,87,50,121,22,74,68,110,29,6,78,...,13,1,15,27,30,32,42,9,44,18,10,27,9,29,25,33,12,9,17,4,45,0,11,49,34,19,13,7,12,16,5,14,17,13,11,15,10,11,14,13
29033047,2017,2,29,2900207,2,2,2,1,48,49,81,21,68,33,67,36,42,57,99,4,95,7,87,15,100,2,78,23,23,79,36,62,76,24,64,35,77,20,4,55,...,3,2,17,19,25,24,14,11,15,17,12,18,5,24,14,29,12,6,10,1,30,1,8,28,26,14,9,9,7,14,4,13,8,4,14,6,12,3,10,6
29446740,2017,2,29,2900306,2,2,1,1,34,65,90,11,82,19,76,25,54,45,100,1,94,6,83,16,96,3,70,28,29,71,27,68,82,14,58,40,82,13,4,60,...,6,0,9,22,26,24,19,1,20,12,14,33,9,12,8,17,17,11,20,1,25,0,4,31,32,19,9,3,3,10,4,12,9,13,7,10,10,8,7,4
61206519,2017,2,29,2900355,2,2,1,1,21,58,58,22,51,29,51,27,35,43,76,3,69,10,62,17,73,6,51,28,12,67,18,56,64,14,33,42,66,9,5,52,...,2,1,8,21,27,12,11,11,30,6,10,6,5,12,23,28,9,4,2,0,11,0,6,21,18,11,10,2,10,6,5,9,10,9,5,9,3,7,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61218410,2017,2,29,6307661,2,2,1,1,56,104,150,10,130,31,136,22,90,67,154,5,145,15,134,23,157,2,110,50,30,128,19,138,117,41,82,76,134,17,8,89,...,11,1,23,39,59,25,13,3,16,18,18,64,20,22,6,23,18,24,41,3,44,0,11,68,46,13,10,7,6,13,17,17,15,17,15,10,15,9,12,12
61227355,2017,2,29,6307663,2,4,1,1,4,9,13,0,13,0,12,1,10,2,13,0,13,0,12,1,13,0,10,3,1,12,1,11,10,3,5,8,13,0,0,11,...,0,0,1,3,5,3,1,0,1,0,1,5,6,0,0,1,0,1,8,2,1,0,0,8,4,1,0,0,0,0,0,0,1,2,2,1,2,3,1,1
61238810,2017,2,29,6307691,2,4,1,1,8,12,20,0,20,0,20,0,16,4,20,0,18,2,19,1,20,0,17,3,3,17,1,19,15,5,11,9,18,1,1,13,...,4,1,2,6,8,3,0,0,0,0,1,11,7,1,0,0,3,3,8,5,1,0,0,11,8,1,0,0,0,2,2,1,3,1,0,3,2,3,2,0


**Pré-processamento da base de dados pessoais**

In [ ]:
#Função que verifica se o ID de uma escola é inválido
#Segundo o Dicionário de Dados da base o SAEB 6 - Os códigos dos Municípios e 
#das Escolas que começam com o dígito "6" são máscaras, isto é, são códigos fictícios.
def retira_id_ficticio(id_escola):
    if id_escola[0] == "6":
        return True
    else:
        return False

In [ ]:
#Se retirou as escolas fictícias
for escola in dados_pessoais.index:
    if (retira_id_ficticio(str(escola))):
        dados_pessoais.drop(escola, inplace=True)
        

dados_pessoais

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_SITUACAO_CENSO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,...,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_B,TX_RESP_Q016_C,TX_RESP_Q016_D,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_C,TX_RESP_Q019_D,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_C,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_C,TX_RESP_Q004_D,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
ID_ESCOLA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
29211590,2017,2,29,2900108,2,2,1,1,36,36,65,7,58,14,63,6,47,20,68,0,65,3,62,6,66,1,49,19,22,44,21,43,44,23,46,21,63,7,2,49,...,6,1,8,16,29,17,1,6,23,14,5,13,4,7,8,21,10,8,8,2,10,0,8,25,15,8,4,1,8,9,5,10,3,7,5,5,3,6,6,7
29032539,2017,2,29,2900207,2,2,1,1,52,95,124,23,107,39,95,53,72,73,145,2,137,9,133,13,142,4,110,36,35,107,87,50,121,22,74,68,110,29,6,78,...,13,1,15,27,30,32,42,9,44,18,10,27,9,29,25,33,12,9,17,4,45,0,11,49,34,19,13,7,12,16,5,14,17,13,11,15,10,11,14,13
29033047,2017,2,29,2900207,2,2,2,1,48,49,81,21,68,33,67,36,42,57,99,4,95,7,87,15,100,2,78,23,23,79,36,62,76,24,64,35,77,20,4,55,...,3,2,17,19,25,24,14,11,15,17,12,18,5,24,14,29,12,6,10,1,30,1,8,28,26,14,9,9,7,14,4,13,8,4,14,6,12,3,10,6
29446740,2017,2,29,2900306,2,2,1,1,34,65,90,11,82,19,76,25,54,45,100,1,94,6,83,16,96,3,70,28,29,71,27,68,82,14,58,40,82,13,4,60,...,6,0,9,22,26,24,19,1,20,12,14,33,9,12,8,17,17,11,20,1,25,0,4,31,32,19,9,3,3,10,4,12,9,13,7,10,10,8,7,4
29236606,2017,2,29,2900504,2,2,1,1,33,64,85,17,76,26,66,34,47,53,100,2,88,13,82,19,99,3,79,23,16,84,26,74,74,27,48,54,86,11,4,68,...,2,0,5,24,25,19,27,8,29,15,7,19,4,20,14,30,14,1,8,1,34,0,11,29,28,16,11,5,2,7,10,9,4,8,9,8,10,10,14,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,2017,2,29,2933307,2,2,1,1,25,34,52,10,41,20,45,16,33,27,61,1,52,9,53,9,60,1,39,22,19,40,5,54,48,11,43,17,36,21,4,39,...,6,1,12,20,17,9,3,3,10,10,10,14,4,9,6,12,10,9,10,0,15,0,2,19,16,11,4,3,7,7,5,5,6,3,5,1,6,8,8,6
29433410,2017,2,29,2933307,2,2,1,1,54,54,103,8,83,27,103,9,71,42,111,3,99,15,88,26,111,2,69,45,44,69,20,91,86,28,55,57,93,16,3,63,...,5,1,10,23,37,25,15,4,22,18,19,27,8,12,4,27,22,13,24,3,21,0,4,34,32,28,4,4,6,5,6,9,11,8,11,16,5,8,8,16
29456487,2017,2,29,2933307,2,2,2,1,36,61,79,18,68,28,64,28,48,44,91,3,82,12,79,15,90,4,54,40,14,78,30,60,61,31,53,37,80,11,2,64,...,3,1,6,19,22,26,23,9,42,16,9,9,1,11,15,43,5,6,4,0,22,0,0,20,30,21,11,7,8,12,9,11,9,8,9,5,8,6,7,3


In [ ]:
#Importando variáveis com informações das escolas e professores
#base_escola_se = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/ESCOLAS2017-SE.csv')

##base_escola_se = pd.read_csv('/content/drive/MyDrive/TCC/ESCOLAS2017-SE.csv')

#base_escola_se.set_index('CO_ENTIDADE', inplace=True)

#base_escola_al = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/ESCOLAS2017-AL.csv')

#base_escola_al = pd.read_csv('/content/drive/MyDrive/TCC/ESCOLAS2017-AL.csv')

#base_escola_al.set_index('CO_ENTIDADE', inplace=True)

#base_escola_ba = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/ESCOLAS2017-BA.csv')

base_escola_ba = pd.read_csv('/content/drive/MyDrive/TCC/ESCOLAS2017-BA.csv')

base_escola_ba.rename(columns={'Unnamed: 0': 'CO_ENTIDADE'}, inplace=True)

base_escola_ba.set_index('CO_ENTIDADE', inplace=True)

#dados_escola = pd.merge(base_escola_se, base_escola_al, how = 'outer')

#dados_escola = pd.concat([dados_escola, base_escola_ba], join='outer')

dados_escola = base_escola_ba

In [ ]:
#dados_escola.set_index('CO_ENTIDADE', inplace=True)

dados_escola

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,...,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,asQtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdedocentesCLT,QtdeDocentesComLicenciatura,QtdeAlunosTotal,Ideb-2017
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,
29433410,2017,EE - COLEGIO ESTADUAL DOM CLIMERIO ALMEIDA DE ...,20.0,1,06/02/2017,22/12/2017,2,2906,29028,29,...,214,0,0,214,0,0,0,210,822,3.3
29208149,2017,EE - COLEGIO ESTADUAL ARISTIDES SILVA - DISTRI...,1.0,1,06/02/2017,13/12/2017,2,2906,29022,29,...,39,0,0,15,24,0,0,30,214,4.1
29526604,2017,EE - COLEGIO ESTADUAL NOSSA SENHORA APARECIDA,11.0,1,06/02/2017,13/12/2017,2,2901,29001,29,...,164,0,0,14,150,0,0,158,727,2.9
29424348,2017,EE - COLEGIO ESTADUAL ANTONIO FIGUEREDO,13.0,1,06/02/2017,13/12/2017,2,2906,29026,29,...,171,0,0,112,59,0,0,171,707,3.6
29378052,2017,CENTRO EDUCACIONAL MONTEIRO LOBATO,13.0,1,06/02/2017,30/11/2017,2,2906,29027,29,...,176,0,0,0,0,0,0,121,911,6.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29223687,2017,EE - COLEGIO ESTADUAL EDIVALDO BOAVENTURA - PO...,9.0,1,06/02/2017,13/12/2017,2,2906,29024,29,...,184,0,0,72,112,0,0,184,797,2.9
29035007,2017,EE - COLEGIO ESTADUAL POLIVALENTE DE PAULO AFONSO,24.0,1,06/02/2017,13/12/2017,2,2902,29005,29,...,332,0,1,302,30,0,0,289,1512,3.9
29242290,2017,EE - INSTITUTO DE EDUCACAO ANISIO TEIXEIRA,13.0,1,06/02/2017,13/12/2017,2,2906,29026,29,...,483,0,0,444,39,0,0,468,2009,4.0


In [ ]:
col_ideb = dados_escola['Ideb-2017']

dados_escola.drop('Ideb-2017', axis=1, inplace=True)

In [ ]:
col_ideb

CO_ENTIDADE
29433410    3.3
29208149    4.1
29526604    2.9
29424348    3.6
29378052    6.4
           ... 
29223687    2.9
29035007    3.9
29242290    4.0
29237645    3.7
29337941    2.5
Name: Ideb-2017, Length: 431, dtype: float64

In [ ]:
dados_escola

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_CATEGORIA_ESCOLA_PRIVADA,IN_CONVENIADA_PP,TP_CONVENIO_PODER_PUBLICO,IN_MANT_ESCOLA_PRIVADA_EMP,IN_MANT_ESCOLA_PRIVADA_ONG,IN_MANT_ESCOLA_PRIVADA_SIND,IN_MANT_ESCOLA_PRIVADA_SIST_S,IN_MANT_ESCOLA_PRIVADA_S_FINS,CO_ESCOLA_SEDE_VINCULADA,CO_IES_OFERTANTE,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,TP_OCUPACAO_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,...,IN_COMUM_CRECHE,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentesDivSocCulta,QtdeDocentesAtendEsp,QtdeDocentesTotal,QtdeTurmasProf,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,asQtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdedocentesCLT,QtdeDocentesComLicenciatura,QtdeAlunosTotal
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
29433410,2017,EE - COLEGIO ESTADUAL DOM CLIMERIO ALMEIDA DE ...,20.0,1,06/02/2017,22/12/2017,2,2906,29028,29,2933307,293330705,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,208,14,4,6,7,0,0,214,214,214,0,0,214,0,0,0,210,822
29208149,2017,EE - COLEGIO ESTADUAL ARISTIDES SILVA - DISTRI...,1.0,1,06/02/2017,13/12/2017,2,2906,29022,29,2914109,291410905,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,9,0,0,39,39,39,0,0,15,24,0,0,30,214
29526604,2017,EE - COLEGIO ESTADUAL NOSSA SENHORA APARECIDA,11.0,1,06/02/2017,13/12/2017,2,2901,29001,29,2911105,291110505,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,111,6,0,47,15,0,0,164,164,164,0,0,14,150,0,0,158,727
29424348,2017,EE - COLEGIO ESTADUAL ANTONIO FIGUEREDO,13.0,1,06/02/2017,13/12/2017,2,2906,29026,29,2912004,291200405,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,130,13,0,37,0,0,0,171,171,171,0,0,112,59,0,0,171,707
29378052,2017,CENTRO EDUCACIONAL MONTEIRO LOBATO,13.0,1,06/02/2017,30/11/2017,2,2906,29027,29,2904605,290460505,4,1,1,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,5,0,93,0,0,0,176,176,176,0,0,0,0,0,0,121,911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29223687,2017,EE - COLEGIO ESTADUAL EDIVALDO BOAVENTURA - PO...,9.0,1,06/02/2017,13/12/2017,2,2906,29024,29,2904308,290430805,2,2,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,78,8,0,89,0,0,0,184,184,184,0,0,72,112,0,0,184,797
29035007,2017,EE - COLEGIO ESTADUAL POLIVALENTE DE PAULO AFONSO,24.0,1,06/02/2017,13/12/2017,2,2902,29005,29,2924009,292400905,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0

#### Pré-processamento da base das escolas

In [ ]:
#Retirando valores ausentes
filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

# retorna o número de valores ausentes para cada coluna
dados_escola[listOfColumnNames].isnull().sum()

CO_ORGAO_REGIONAL    7
dtype: int64

In [ ]:
#Retirando colunas que apresentam mais de 60% do valores nulos
dados_escola.dropna(thresh=len(dados_escola)*0.6, axis=1, inplace=True)

#Preenchendo valores ausentes
dados_escola.fillna(0, inplace=True)

filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

# retorna o número de valores ausentes para cada coluna
dados_escola[listOfColumnNames].isnull().sum()

CO_ORGAO_REGIONAL    0
dtype: int64

In [ ]:
dados_escola[listOfColumnNames] = dados_escola[listOfColumnNames].astype(int)

In [ ]:
dados_escola.insert(len(dados_escola.columns), 'Ideb-2017', col_ideb)
#['Ideb-2017'] = col_ideb

In [ ]:
dados_escola

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_CATEGORIA_ESCOLA_PRIVADA,IN_CONVENIADA_PP,TP_CONVENIO_PODER_PUBLICO,IN_MANT_ESCOLA_PRIVADA_EMP,IN_MANT_ESCOLA_PRIVADA_ONG,IN_MANT_ESCOLA_PRIVADA_SIND,IN_MANT_ESCOLA_PRIVADA_SIST_S,IN_MANT_ESCOLA_PRIVADA_S_FINS,CO_ESCOLA_SEDE_VINCULADA,CO_IES_OFERTANTE,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,TP_OCUPACAO_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,...,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentesDivSocCulta,QtdeDocentesAtendEsp,QtdeDocentesTotal,QtdeTurmasProf,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,asQtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdedocentesCLT,QtdeDocentesComLicenciatura,QtdeAlunosTotal,Ideb-2017
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
29433410,2017,EE - COLEGIO ESTADUAL DOM CLIMERIO ALMEIDA DE ...,20,1,06/02/2017,22/12/2017,2,2906,29028,29,2933307,293330705,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,208,14,4,6,7,0,0,214,214,214,0,0,214,0,0,0,210,822,3.3
29208149,2017,EE - COLEGIO ESTADUAL ARISTIDES SILVA - DISTRI...,1,1,06/02/2017,13/12/2017,2,2906,29022,29,2914109,291410905,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,9,0,0,39,39,39,0,0,15,24,0,0,30,214,4.1
29526604,2017,EE - COLEGIO ESTADUAL NOSSA SENHORA APARECIDA,11,1,06/02/2017,13/12/2017,2,2901,29001,29,2911105,291110505,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,111,6,0,47,15,0,0,164,164,164,0,0,14,150,0,0,158,727,2.9
29424348,2017,EE - COLEGIO ESTADUAL ANTONIO FIGUEREDO,13,1,06/02/2017,13/12/2017,2,2906,29026,29,2912004,291200405,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,130,13,0,37,0,0,0,171,171,171,0,0,112,59,0,0,171,707,3.6
29378052,2017,CENTRO EDUCACIONAL MONTEIRO LOBATO,13,1,06/02/2017,30/11/2017,2,2906,29027,29,2904605,290460505,4,1,1,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,5,0,93,0,0,0,176,176,176,0,0,0,0,0,0,121,911,6.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29223687,2017,EE - COLEGIO ESTADUAL EDIVALDO BOAVENTURA - PO...,9,1,06/02/2017,13/12/2017,2,2906,29024,29,2904308,290430805,2,2,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,78,8,0,89,0,0,0,184,184,184,0,0,72,112,0,0,184,797,2.9
29035007,2017,EE - COLEGIO ESTADUAL POLIVALENTE DE PAULO AFONSO,24,1,06/02/2017,13/12/2017,2,2902,29005,29,2924009,292400905,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,282,6

In [ ]:
#Colunas categóricas
filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.object]

listOfColumnNames = list(filteredColumns.index)

dados_escola[listOfColumnNames]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


,NO_ENTIDADE,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO
CO_ENTIDADE,,,
29433410,EE - COLEGIO ESTADUAL DOM CLIMERIO ALMEIDA DE ...,06/02/2017,22/12/2017
29208149,EE - COLEGIO ESTADUAL ARISTIDES SILVA - DISTRI...,06/02/2017,13/12/2017
29526604,EE - COLEGIO ESTADUAL NOSSA SENHORA APARECIDA,06/02/2017,13/12/2017
29424348,EE - COLEGIO ESTADUAL ANTONIO FIGUEREDO,06/02/2017,13/12/2017
29378052,CENTRO EDUCACIONAL MONTEIRO LOBATO,06/02/2017,30/11/2017
...,...,...,...
29223687,EE - COLEGIO ESTADUAL EDIVALDO BOAVENTURA - PO...,06/02/2017,13/12/2017
29035007,EE - COLEGIO ESTADUAL POLIVALENTE DE PAULO AFONSO,06/02/2017,13/12/2017
29242290,EE - INSTITUTO DE EDUCACAO ANISIO TEIXEIRA,06/02/2017,13/12/2017


In [ ]:
#Tratando datas, convertendo para timestamp

inicio = pd.to_datetime(dados_escola['DT_ANO_LETIVO_INICIO'], infer_datetime_format = True)
termino = pd.to_datetime(dados_escola['DT_ANO_LETIVO_TERMINO'], infer_datetime_format = True)

dados_escola['DT_ANO_LETIVO_INICIO'] = inicio.values.astype (np.int64)
dados_escola['DT_ANO_LETIVO_TERMINO'] = termino.values.astype (np.int64)

In [ ]:
dados_escola.drop('NO_ENTIDADE', axis=1, inplace=True)

In [ ]:
dados_escola.columns.to_list()

['NU_ANO_CENSO',
 'CO_ORGAO_REGIONAL',
 'TP_SITUACAO_FUNCIONAMENTO',
 'DT_ANO_LETIVO_INICIO',
 'DT_ANO_LETIVO_TERMINO',
 'CO_REGIAO',
 'CO_MESORREGIAO',
 'CO_MICRORREGIAO',
 'CO_UF',
 'CO_MUNICIPIO',
 'CO_DISTRITO',
 'TP_DEPENDENCIA',
 'TP_LOCALIZACAO',
 'TP_CATEGORIA_ESCOLA_PRIVADA',
 'IN_CONVENIADA_PP',
 'TP_CONVENIO_PODER_PUBLICO',
 'IN_MANT_ESCOLA_PRIVADA_EMP',
 'IN_MANT_ESCOLA_PRIVADA_ONG',
 'IN_MANT_ESCOLA_PRIVADA_SIND',
 'IN_MANT_ESCOLA_PRIVADA_SIST_S',
 'IN_MANT_ESCOLA_PRIVADA_S_FINS',
 'CO_ESCOLA_SEDE_VINCULADA',
 'CO_IES_OFERTANTE',
 'TP_REGULAMENTACAO',
 'IN_LOCAL_FUNC_PREDIO_ESCOLAR',
 'TP_OCUPACAO_PREDIO_ESCOLAR',
 'IN_LOCAL_FUNC_SALAS_EMPRESA',
 'IN_LOCAL_FUNC_SOCIOEDUCATIVO',
 'IN_LOCAL_FUNC_UNID_PRISIONAL',
 'IN_LOCAL_FUNC_PRISIONAL_SOCIO',
 'IN_LOCAL_FUNC_TEMPLO_IGREJA',
 'IN_LOCAL_FUNC_CASA_PROFESSOR',
 'IN_LOCAL_FUNC_GALPAO',
 'TP_OCUPACAO_GALPAO',
 'IN_LOCAL_FUNC_SALAS_OUTRA_ESC',
 'IN_LOCAL_FUNC_OUTROS',
 'IN_PREDIO_COMPARTILHADO',
 'IN_AGUA_FILTRADA',
 'IN_AGUA_RE

In [ ]:
dados_pessoais

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_SITUACAO_CENSO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,...,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_B,TX_RESP_Q016_C,TX_RESP_Q016_D,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_C,TX_RESP_Q019_D,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_C,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_C,TX_RESP_Q004_D,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
ID_ESCOLA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
29211590,2017,2,29,2900108,2,2,1,1,36,36,65,7,58,14,63,6,47,20,68,0,65,3,62,6,66,1,49,19,22,44,21,43,44,23,46,21,63,7,2,49,...,6,1,8,16,29,17,1,6,23,14,5,13,4,7,8,21,10,8,8,2,10,0,8,25,15,8,4,1,8,9,5,10,3,7,5,5,3,6,6,7
29032539,2017,2,29,2900207,2,2,1,1,52,95,124,23,107,39,95,53,72,73,145,2,137,9,133,13,142,4,110,36,35,107,87,50,121,22,74,68,110,29,6,78,...,13,1,15,27,30,32,42,9,44,18,10,27,9,29,25,33,12,9,17,4,45,0,11,49,34,19,13,7,12,16,5,14,17,13,11,15,10,11,14,13
29033047,2017,2,29,2900207,2,2,2,1,48,49,81,21,68,33,67,36,42,57,99,4,95,7,87,15,100,2,78,23,23,79,36,62,76,24,64,35,77,20,4,55,...,3,2,17,19,25,24,14,11,15,17,12,18,5,24,14,29,12,6,10,1,30,1,8,28,26,14,9,9,7,14,4,13,8,4,14,6,12,3,10,6
29446740,2017,2,29,2900306,2,2,1,1,34,65,90,11,82,19,76,25,54,45,100,1,94,6,83,16,96,3,70,28,29,71,27,68,82,14,58,40,82,13,4,60,...,6,0,9,22,26,24,19,1,20,12,14,33,9,12,8,17,17,11,20,1,25,0,4,31,32,19,9,3,3,10,4,12,9,13,7,10,10,8,7,4
29236606,2017,2,29,2900504,2,2,1,1,33,64,85,17,76,26,66,34,47,53,100,2,88,13,82,19,99,3,79,23,16,84,26,74,74,27,48,54,86,11,4,68,...,2,0,5,24,25,19,27,8,29,15,7,19,4,20,14,30,14,1,8,1,34,0,11,29,28,16,11,5,2,7,10,9,4,8,9,8,10,10,14,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,2017,2,29,2933307,2,2,1,1,25,34,52,10,41,20,45,16,33,27,61,1,52,9,53,9,60,1,39,22,19,40,5,54,48,11,43,17,36,21,4,39,...,6,1,12,20,17,9,3,3,10,10,10,14,4,9,6,12,10,9,10,0,15,0,2,19,16,11,4,3,7,7,5,5,6,3,5,1,6,8,8,6
29433410,2017,2,29,2933307,2,2,1,1,54,54,103,8,83,27,103,9,71,42,111,3,99,15,88,26,111,2,69,45,44,69,20,91,86,28,55,57,93,16,3,63,...,5,1,10,23,37,25,15,4,22,18,19,27,8,12,4,27,22,13,24,3,21,0,4,34,32,28,4,4,6,5,6,9,11,8,11,16,5,8,8,16
29456487,2017,2,29,2933307,2,2,2,1,36,61,79,18,68,28,64,28,48,44,91,3,82,12,79,15,90,4,54,40,14,78,30,60,61,31,53,37,80,11,2,64,...,3,1,6,19,22,26,23,9,42,16,9,9,1,11,15,43,5,6,4,0,22,0,0,20,30,21,11,7,8,12,9,11,9,8,9,5,8,6,7,3


### Unindo as bases de dados 

Foi realizado join nos dois datasets, nesse momento se percebeu que não existem variáveis escolas particulares na base derivada do SAEB

In [ ]:
dataset = dados_pessoais.join(dados_escola, how="inner")

In [ ]:
dataset

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_SITUACAO_CENSO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,...,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentesDivSocCulta,QtdeDocentesAtendEsp,QtdeDocentesTotal,QtdeTurmasProf,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,asQtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdedocentesCLT,QtdeDocentesComLicenciatura,QtdeAlunosTotal,Ideb-2017
29211590,2017,2,29,2900108,2,2,1,1,36,36,65,7,58,14,63,6,47,20,68,0,65,3,62,6,66,1,49,19,22,44,21,43,44,23,46,21,63,7,2,49,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,105,0,0,0,3,0,0,116,116,116,0,0,110,6,0,0,105,287,3.7
29032539,2017,2,29,2900207,2,2,1,1,52,95,124,23,107,39,95,53,72,73,145,2,137,9,133,13,142,4,110,36,35,107,87,50,121,22,74,68,110,29,6,78,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,95,0,0,37,3,0,0,147,147,147,0,0,95,52,0,0,127,685,3.5
29033047,2017,2,29,2900207,2,2,2,1,48,49,81,21,68,33,67,36,42,57,99,4,95,7,87,15,100,2,78,23,23,79,36,62,76,24,64,35,77,20,4,55,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,96,0,0,6,5,0,0,108,108,108,0,0,69,39,0,0,102,442,3.1
29446740,2017,2,29,2900306,2,2,1,1,34,65,90,11,82,19,76,25,54,45,100,1,94,6,83,16,96,3,70,28,29,71,27,68,82,14,58,40,82,13,4,60,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,74,12,0,78,0,0,0,154,154,154,0,0,69,85,0,0,129,689,2.5
29236606,2017,2,29,2900504,2,2,1,1,33,64,85,17,76,26,66,34,47,53,100,2,88,13,82,19,99,3,79,23,16,84,26,74,74,27,48,54,86,11,4,68,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,64,0,0,28,0,0,0,130,130,130,0,0,57,73,0,0,104,551,3.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,2017,2,29,2933307,2,2,1,1,25,34,52,10,41,20,45,16,33,27,61,1,52,9,53,9,60,1,39,22,19,40,5,54,48,11,43,17,36,21,4,39,...,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,167,4,0,10,12,0,0,182,182,182,0,0,167,15,0,0,170,877,2.9
29433410,2017,2,29,2933307,2,2,1,1,54,54,103,8,83,27,103,9,71,42,111,3,99,15,88,26,111,2,69,45,44,69,20,91,86,28,55,57,93,16,3,63,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,208,14,4,6,7,0,0,214,214,214,0,0,214,0,0,0,210,822,3.3
29456487,2017,2,29,2933307,2,2,2,1,36,61,79,18,68,28,64,28,48,44,91,3,82,12,79,15,90,4,54,40,14,78,30,60,61,31,53,37,80,11,2,64,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,125,7,0,19,4,0,0,151,151,151,0,0,99,52,0,0,138,370,3.2
29457491,2017,2,29,2933307,2,2,2,1,27,33,39,16,35,20,37,19,27,31,55,2,47,10,48,9,54,3,40,14,15,41,21,34,47,11,32,22,47,12,0,34,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,9,0,0,0,0,0,68,68,68,0,0,60,8,0,0,68,268,3.1


In [ ]:
dados_escola.dtypes[dados_escola.dtypes == np.float64]

Ideb-2017    float64
dtype: object

In [ ]:
dataset['ID_DEPENDENCIA_ADM'].unique()

array([2, 3, 1])

In [ ]:
#dataset.to_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa Ideb/variaveis_derivadas_escola_estudante_5.0.csv')
dataset.to_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_escola_estudante_5.0.csv')

### Discretização dos rótulos, nota do Ideb

**Discretização:** É o processo de transferência de funções contínuas, modelos, variáveis e equações em contrapartes discretas.

In [ ]:
# São três classes média, acima da média e abaixo da média
def discretizacao(ideb, mean_ideb):
    classe = []
    for nota in ideb['Ideb-2017']:
        if (mean_ideb == nota):
            classe.append('média')
        elif (mean_ideb < nota):
            classe.append('acima da média')
        else:
            classe.append('abaixo da média')
    return classe

In [ ]:
#discretizando as notas do Ideb

Ideb = dataset[['Ideb-2017']]

mean_Ideb = round(Ideb.mean(),1)

classe_ideb = discretizacao(Ideb, mean_Ideb['Ideb-2017'])

Ideb.insert(1, 'classe-Ideb', classe_ideb)

Ideb

,Ideb-2017,classe-Ideb
29211590,3.7,acima da média
29032539,3.5,acima da média
29033047,3.1,média
29446740,2.5,abaixo da média
29236606,3.1,média
...,...,...
29409500,2.9,abaixo da média
29433410,3.3,acima da média
29456487,3.2,acima da média
29457491,3.1,média


In [ ]:
#Variavel a ser prevista
y = Ideb['classe-Ideb']

**Seleção de Atributos:**  É um passo da fase de pré-processamento que tem a finalidade de selecionar o melhor subconjunto de atributos que pode substituir o conjunto de dados original. 

https://www.analyticsvidhya.com/blog/2016/12/introduction-to-feature-selection-methods-with-an-example-or-how-to-select-the-right-variables/#:~:text=Embedded%20methods%20combine%20the%20qualities,penalization%20functions%20to%20reduce%20overfitting.

Abordagens:

- Filtro: a relevância dos atributos é avaliada independentemente – não
envolve o uso de um método de AM ;

- Wrapper (Envólucro): Avalia o subconjunto de atributos arti culado ao método de AM que realizará a indução do conceito;

- Embedded (Embutidos): Os métodos incorporados combinam as qualidades dos métodos de filtro e wrapper. Ele é implementado por algoritmos que possuem seus próprios métodos de seleção de recursos integrados.

In [ ]:
#Filter based
# f_classif: ANOVA F-value between label/feature for classification tasks.
#ANOVA stands for Analysis of variance. It is similar to LDA except for the fact that it is operated using one or more categorical 
#independent features and one continuous dependent feature. It provides a statistical test 
#of whether the means of several groups are equal or not.
def selecao_atributos(X, y, quant):
    select = SelectKBest(score_func=f_classif, k=quant)
    #select = SelectKBest(score_func=chi2, k=quant)
    z = select.fit_transform(X, y)
    
    filter = select.get_support(indices=True)
    features = array(X.columns)
    
    return X.iloc[:, filter]

In [ ]:
#Wrapper-based
#from sklearn.cluster import KMeans

def wrapper_based(X, X_transform, y, quant):

  #select_features = RFE(estimator=KMeans(n_clusters=3), n_features_to_select=100)
  select_features = RFE(estimator=SVC(kernel="linear"), n_features_to_select=100)
  '''select_features = RFECV(
      estimator=SVC(kernel="linear"),
      min_features_to_select= quant,
      cv = 5
  )'''
  select_features.fit(X_transform, y)

  filter = select_features.get_support(indices=True)
  return X.iloc[:, filter]

In [ ]:
# Embedded-based

def embedded_based(X, X_transform, y, quant):

  select_features = SelectFromModel(LogisticRegression(), max_features=quant)
  #select_features = SelectFromModel(DecisionTreeClassifier(), max_features=quant)
  #select_features = SelectFromModel(SVC(kernel="rbf"), max_features=quant)
  #select_features = SelectFromModel(RandomForestClassifier(), max_features=quant)
  select_features.fit(X_transform, y)

  filter = select_features.get_support(indices=True)

  return X.iloc[:, filter]

**Métricas de Avaliação**


In [ ]:
def acuracia(resultados, classes):
    #resultados = cross_val_predict(modelo, textos, classes, cv=10)  
    return 'Acurácia do modelo: {} %'.format(round(metrics.accuracy_score(classes, resultados)*100, 2))

**Normalização:** é mudar os valores das colunas numéricas no conjunto de dados para usar uma escala comum, sem distorcer as diferenças nos intervalos de valores nem perder informações.

- **Normalização MinMax:** Para cada recurso, o valor mínimo desse recurso é transformado em 0, o valor máximo é transformado em 1 e todos os outros valores são transformados em um decimal entre 0 e 1.

In [ ]:
def minmax(data):
    scaler = MinMaxScaler()
    data_scaler = scaler.fit_transform(data)
    return data_scaler

## Criação dos modelos

**Serão feitas três análises:** Apenas com as variáveis socioeconômicas, com as variáveis de escolas e professores e depois com o dataset completo

Para cada uma serão criados modelos com todos os atributos e também com seleção de atributos, a fim de verificar o de maior acurácia

In [ ]:
def inicializar_modelos():
    naive = MultinomialNB()
    random = RandomForestClassifier(n_estimators=150)
    decision_tree = DecisionTreeClassifier()
    svm = SVC(kernel="linear")
    r_logistic = LogisticRegression(max_iter=250)
    knn = KNeighborsClassifier(n_neighbors=3)
    return naive, random, decision_tree, svm, r_logistic, knn

#### -Primeira análise: variáveis socioeconômicas derivadas da prova SAEB**

In [ ]:
#Foi retirada colunas do 'ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF'
columns = dataset.columns[3:234]

columns

Index(['ID_MUNICIPIO', 'ID_AREA', 'ID_DEPENDENCIA_ADM', 'ID_LOCALIZACAO',
       'IN_SITUACAO_CENSO', 'TX_RESP_Q001_A', 'TX_RESP_Q001_B',
       'TX_RESP_Q020_A', 'TX_RESP_Q020_B', 'TX_RESP_Q021_A',
       ...
       'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D',
       'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H',
       'TX_RESP_Q003_I', 'TX_RESP_Q003_J'],
      dtype='object', length=231)

In [ ]:
X =  dataset[columns]

In [ ]:
#Questões de sexo, raça, ano, mes de nascimento, conformidade com censo
question = ['TX_RESP_Q001_A', 'TX_RESP_Q001_B', 'TX_RESP_Q002_A', 'TX_RESP_Q002_B', 'TX_RESP_Q002_C', 'TX_RESP_Q002_D', 
            'TX_RESP_Q002_E', 'TX_RESP_Q002_F', 'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D', 
            'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H', 'TX_RESP_Q003_I', 'TX_RESP_Q003_J', 
            'TX_RESP_Q004_A', 'TX_RESP_Q004_B', 'TX_RESP_Q004_C', 'TX_RESP_Q004_D', 
            'TX_RESP_Q004_E', 'TX_RESP_Q004_F', 'TX_RESP_Q004_G', 'TX_RESP_Q004_H', 'IN_SITUACAO_CENSO']

delete_at = ['IN_SITUACAO_CENSO']

X.drop(delete_at, axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
filteredColumns = X.dtypes[X.dtypes == np.float]

listOfColumnNames = list(filteredColumns.index)

X[listOfColumnNames]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


""
29211590
29032539
29033047
29446740
29236606
...
29409500
29433410
29456487
29457491


**Divisão entre conjunto de treino e teste**

In [ ]:
X_scaler = minmax(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 67.09 %
Naive Bayes: Acurácia do modelo: 63.29 %
Decision Tree: Acurácia do modelo: 58.23 %
SVM: Acurácia do modelo: 64.56 %
Logistic Regression: Acurácia do modelo: 64.56 %
KNN: Acurácia do modelo: 48.1 %


**-Primeira Análise: Aplicando Seleção de Atributos (Filtragem)**

In [ ]:
X_selection = selecao_atributos(X, y, 100)

X_selection

,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q020_B,TX_RESP_Q024_B,TX_RESP_Q027_B,TX_RESP_Q030_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q018_B,TX_RESP_Q022_B,TX_RESP_Q022_C,TX_RESP_Q033_C,TX_RESP_Q035_A,TX_RESP_Q035_C,TX_RESP_Q040_B,TX_RESP_Q040_C,TX_RESP_Q041_A,TX_RESP_Q041_B,TX_RESP_Q041_C,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q046_B,TX_RESP_Q047_B,TX_RESP_Q049_A,TX_RESP_Q050_B,TX_RESP_Q054_C,TX_RESP_Q055_A,TX_RESP_Q055_B,TX_RESP_Q055_C,TX_RESP_Q056_A,TX_RESP_Q057_A,TX_RESP_Q057_B,TX_RESP_Q058_A,TX_RESP_Q039_A,TX_RESP_Q039_B,TX_RESP_Q039_C,TX_RESP_Q045_B,TX_RESP_Q060_C,...,TX_RESP_Q013_B,TX_RESP_Q013_C,TX_RESP_Q013_D,TX_RESP_Q013_E,TX_RESP_Q014_A,TX_RESP_Q014_C,TX_RESP_Q014_D,TX_RESP_Q014_E,TX_RESP_Q015_A,TX_RESP_Q015_B,TX_RESP_Q017_C,TX_RESP_Q017_D,TX_RESP_Q017_E,TX_RESP_Q036_A,TX_RESP_Q036_B,TX_RESP_Q036_C,TX_RESP_Q036_E,TX_RESP_Q037_B,TX_RESP_Q002_A,TX_RESP_Q002_E,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_D,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_C,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H
29211590,2,2,1,7,6,0,1,21,43,7,16,3,25,47,0,1,0,46,5,17,1,10,8,48,36,32,3,1,42,17,2,41,6,51,44,11,11,1,2,29,...,33,2,1,0,0,15,1,0,0,1,2,0,0,12,27,2,13,6,30,0,6,1,29,1,6,23,13,4,7,8,8,8,2,10,0,25,8,4,1,8
29032539,2,2,1,23,53,2,4,87,50,29,64,5,56,78,5,5,0,74,16,52,10,11,28,82,77,58,16,2,69,42,3,71,13,93,88,31,25,4,5,74,...,23,7,1,0,6,22,4,1,0,8,2,0,0,43,38,13,16,36,14,3,13,1,30,42,9,44,27,9,29,25,9,17,4,45,0,49,19,13,7,12
29033047,2,2,2,21,36,4,2,36,62,20,42,6,63,52,9,0,3,47,17,36,6,5,22,53,46,39,10,0,47,33,0,57,8,69,48,37,22,2,1,44,...,21,0,0,0,3,8,1,2,0,6,0,0,0,33,21,12,14,17,7,10,3,2,25,14,11,15,18,5,24,14,6,10,1,30,1,28,14,9,9,7
29446740,2,2,1,11,25,1,3,27,68,13,39,2,39,69,5,8,3,54,13,28,5,3,13,67,59,58,8,1,80,10,1,44,12,67,74,39,10,3,1,57,...,28,6,0,1,1,24,4,0,1,5,0,2,1,18,33,9,19,9,8,1,6,0,26,19,1,20,33,9,12,8,11,20,1,25,0,31,19,9,3,3
29236606,2,2,1,17,34,2,3,26,74,11,30,4,44,64,8,3,0,60,10,30,2,7,13,74,60,45,18,0,70,22,1,40,7,78,57,24,11,2,4,65,...,27,1,0,0,1,16,1,0,0,0,0,1,1,25,20,6,18,22,9,1,2,0,25,27,8,29,19,4,20,14,1,8,1,34,0,29,16,11,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,2,2,1,10,16,1,1,5,54,21,21,2,25,39,2,6,2,38,7,12,5,10,8,39,30,40,1,0,39,16,0,29,4,42,41,22,8,1,2,26,...,30,2,1,0,0,13,3,0,0,4,1,0,0,11,22,3,10,10,9,1,6,1,17,3,3,10,14,4,9,6,9,10,0,15,0,19,11,4,3,7
29433410,2,2,1,8,9,3,2,20,91,16,46,5,56,80,2,13,2,61,15,34,3,10,14,85,73,53,7,1,63,38,3,45,8,91,93,36,17,2,0,42,...,64,11,2,0,0,30,4,1,0,0,3,1,0,13,49,5,18,12,21,0,5,1,37,15,4,22,27,8,12,4,13,24,3,21,0,34,28,4,4,6
29456487,2,2,2,18,28,3,4,30,60,11,28,3,43,60,3,2,0,37,18,39,3,6,7,69,56,28,22,0,51,28,0,39,5,74,45,3,64,1,7,16,...,30,2,0,0,0,16,3,0,0,1,2,0,0,23,16,7,14,23,17,0,3,1,22,23,9,42,9,1,11,15,6,4,0,22,0,20,21,11,7,8
29457491,2,2,2,16,19,2,3,21,34,12,24,1,27,38,4,0,0,20,12,24,7,9,15,33,33,27,15,0,35,14,3,17,9,38,20,2,35,3,0,3,...,10,1,1,0,2,5,0,0,1,3,3,1,0,15,13,6,6,15,1,0,1,1,16,8,12,29,3,1,3,16,3,2,0,11,0,4,14,8,6,14


In [ ]:
X_scaler = minmax(X_selection)

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 64.56 %
Naive Bayes: Acurácia do modelo: 64.56 %
Decision Tree: Acurácia do modelo: 49.37 %
SVM: Acurácia do modelo: 63.29 %
Logistic Regression: Acurácia do modelo: 65.82 %
KNN: Acurácia do modelo: 55.7 %


**-Primeira Análise: Aplicando Seleção de Atributos (Embutida)**

In [ ]:
X_embedded = embedded_based(X, X_scaler, y, 100)

X_embedded

,ID_MUNICIPIO,ID_AREA,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q027_B,TX_RESP_Q029_A,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q022_A,TX_RESP_Q022_C,TX_RESP_Q026_A,TX_RESP_Q026_B,TX_RESP_Q026_C,TX_RESP_Q035_C,TX_RESP_Q040_A,TX_RESP_Q040_B,TX_RESP_Q040_C,TX_RESP_Q041_A,TX_RESP_Q041_B,TX_RESP_Q041_C,TX_RESP_Q042_C,TX_RESP_Q046_B,TX_RESP_Q046_C,TX_RESP_Q048_B,TX_RESP_Q049_A,TX_RESP_Q049_B,TX_RESP_Q054_B,TX_RESP_Q055_B,TX_RESP_Q057_B,TX_RESP_Q057_C,TX_RESP_Q039_B,TX_RESP_Q039_C
29211590,2900108,2,1,36,14,63,6,0,62,66,1,19,22,44,23,21,63,7,49,3,39,6,24,0,68,1,0,46,5,17,10,48,3,8,32,21,3,17,51,2,11,1
29032539,2900207,2,1,52,39,95,53,2,133,142,4,36,35,121,22,68,110,29,78,5,86,14,47,5,142,5,0,74,16,52,11,82,5,30,58,55,9,42,93,4,25,4
29033047,2900207,2,2,48,33,67,36,4,87,100,2,23,23,76,24,35,77,20,55,6,65,10,28,9,99,0,3,47,17,36,5,53,5,18,39,37,3,33,69,0,22,2
29446740,2900306,2,1,34,19,76,25,1,83,96,3,28,29,82,14,40,82,13,60,2,67,4,29,5,90,8,3,54,13,28,3,67,1,8,58,21,8,10,67,3,10,3
29236606,2900504,2,1,33,26,66,34,2,82,99,3,23,16,74,27,54,86,11,68,4,46,12,44,8,98,3,0,60,10,30,7,74,4,15,45,41,3,22,78,6,11,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,2933307,2,1,25,20,45,16,1,53,60,1,22,19,48,11,17,36,21,39,2,21,16,25,2,52,6,2,38,7,12,10,39,2,5,40,10,3,16,42,1,8,1
29433410,2933307,2,1,54,27,103,9,3,88,111,2,45,44,86,28,57,93,16,63,5,33,37,43,2,98,13,2,61,15,34,10,85,6,18,53,48,11,38,91,3,17,2
29456487,2933307,2,2,36,28,64,28,3,79,90,4,40,14,61,31,37,80,11,64,3,50,19,26,3,93,2,0,37,18,39,6,69,4,14,28,50,3,28,74,1,64,1
29457491,2933307,2,2,27,20,37,19,2,48,54,3,14,15,47,11,22,47,12,34,1,31,16,10,4,57,0,0,20,12,24,9,33,3,4,27,19,1,14,38,1,35,3


In [ ]:
X_scaler_em = minmax(X_embedded)

X_train, X_test, y_train, y_test = train_test_split(X_scaler_em, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 59.49 %
Naive Bayes: Acurácia do modelo: 64.56 %
Decision Tree: Acurácia do modelo: 59.49 %
SVM: Acurácia do modelo: 70.89 %
Logistic Regression: Acurácia do modelo: 70.89 %
KNN: Acurácia do modelo: 58.23 %


**-Primeira Análise: Aplicando Seleção de Atributos (Wrapper)**

In [ ]:
X_wrapper = wrapper_based(X, X_scaler, y, 100)

X_wrapper

,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,TX_RESP_Q022_B,TX_RESP_Q022_C,TX_RESP_Q026_A,TX_RESP_Q026_B,...,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q046_B,TX_RESP_Q046_C,TX_RESP_Q047_A,TX_RESP_Q047_B,TX_RESP_Q047_C,TX_RESP_Q048_A,TX_RESP_Q048_B,TX_RESP_Q048_C,TX_RESP_Q049_A,TX_RESP_Q049_B,TX_RESP_Q049_C,TX_RESP_Q050_A,TX_RESP_Q050_B,TX_RESP_Q050_C,TX_RESP_Q054_A,TX_RESP_Q054_B,TX_RESP_Q054_C,TX_RESP_Q055_A,TX_RESP_Q055_B,TX_RESP_Q055_C,TX_RESP_Q056_A,TX_RESP_Q056_B,TX_RESP_Q056_C,TX_RESP_Q057_A,TX_RESP_Q057_B,TX_RESP_Q057_C,TX_RESP_Q058_A,TX_RESP_Q058_B,TX_RESP_Q058_C,TX_RESP_Q039_A,TX_RESP_Q039_B,TX_RESP_Q039_C,TX_RESP_Q039_D,TX_RESP_Q045_A,TX_RESP_Q045_B,TX_RESP_Q045_C,TX_RESP_Q045_D
29211590,2900108,2,2,1,36,36,65,7,58,14,63,6,47,20,68,0,65,3,62,6,66,1,49,19,22,44,21,43,44,23,46,21,63,7,2,49,16,3,39,6,...,1,10,8,48,3,20,36,2,48,8,2,32,21,2,57,3,2,58,3,1,42,17,2,41,17,2,6,51,2,44,16,1,11,11,1,45,45,2,20,1
29032539,2900207,2,2,1,52,95,124,23,107,39,95,53,72,73,145,2,137,9,133,13,142,4,110,36,35,107,87,50,121,22,74,68,110,29,6,78,64,5,86,14,...,10,11,28,82,5,31,77,6,92,30,3,58,55,3,109,16,3,114,9,2,69,42,3,71,39,3,13,93,4,88,26,3,31,25,4,88,101,5,38,3
29033047,2900207,2,2,2,48,49,81,21,68,33,67,36,42,57,99,4,95,7,87,15,100,2,78,23,23,79,36,62,76,24,64,35,77,20,4,55,42,6,65,10,...,6,5,22,53,5,29,46,4,74,18,1,39,37,4,76,10,1,91,3,0,47,33,0,57,24,1,8,69,0,48,30,0,37,22,2,42,67,1,33,1
29446740,2900306,2,2,1,34,65,90,11,82,19,76,25,54,45,100,1,94,6,83,16,96,3,70,28,29,71,27,68,82,14,58,40,82,13,4,60,39,2,67,4,...,5,3,13,67,1,19,59,1,84,8,1,58,21,2,77,8,1,80,8,1,80,10,1,44,37,2,12,67,3,74,13,1,39,10,3,46,65,1,31,1
29236606,2900504,2,2,1,33,64,85,17,76,26,66,34,47,53,100,2,88,13,82,19,99,3,79,23,16,84,26,74,74,27,48,54,86,11,4,68,30,4,46,12,...,2,7,13,74,4,26,60,4,71,15,3,45,41,4,78,18,2,92,3,0,70,22,1,40,47,4,7,78,6,57,36,1,24,11,2,63,53,4,43,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,2933307,2,2,1,25,34,52,10,41,20,45,16,33,27,61,1,52,9,53,9,60,1,39,22,19,40,5,54,48,11,43,17,36,21,4,39,21,2,21,16,...,5,10,8,39,2,19,30,1,46,5,1,40,10,2,54,1,2,52,3,0,39,16,0,29,17,2,4,42,1,41,13,1,22,8,1,30,40,2,19,0
29433410,2933307,2,2,1,54,54,103,8,83,27,103,9,71,42,111,3,99,15,88,26,111,2,69,45,44,69,20,91,86,28,55,57,93,16,3,63,46,5,33,37,...,3,10,14,85,6,25,73,5,84,18,5,53,48,5,101,7,1,92,11,1,63,38,3,45,56,4,8,91,3,93,14,2,36,17,2,59,66,0,44,4
29456487,2933307,2,2,2,36,61,79,18,68,28,64,28,48,44,91,3,82,12,79,15,90,4,54,40,14,78,30,60,61,31,53,37,80,11,2,64,28,3,50,19,...,3,6,7,69,4,19,56,3,72,14,1,28,50,2,56,22,1,87,3,0,51,28,0,39,38,3,5,74,1,45,36,1,3,64,1,26,65,7,23,0
29457491,2933307,2,2,2,27,33,39,16,35,20,37,19,27,31,55,2,47,10,48,9,54,3,40,14,15,41,21,34,47,11,32,22,47,12,0,34,24,1,31,16,...,7,9,15,33,3,15,33,0,47,4,1,27,19,1,34,15,0,51,1,0,35,14,3,17,31,1,9,38,1,20,27,1,2,35,3,18,42,0,11,2


In [ ]:
X_scaler_w = minmax(X_wrapper)

X_train, X_test, y_train, y_test = train_test_split(X_scaler_w, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 63.29 %
Naive Bayes: Acurácia do modelo: 65.82 %
Decision Tree: Acurácia do modelo: 51.9 %
SVM: Acurácia do modelo: 70.89 %
Logistic Regression: Acurácia do modelo: 69.62 %
KNN: Acurácia do modelo: 62.03 %


#### -Segunda análise: Análise com o dataset completo

In [ ]:
#Lembrar de tratar datas

'''question = ['TX_RESP_Q001_A', 'TX_RESP_Q001_B', 'TX_RESP_Q002_A', 'TX_RESP_Q002_B', 'TX_RESP_Q002_C', 'TX_RESP_Q002_D', 
            'TX_RESP_Q002_E', 'TX_RESP_Q002_F', 'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D', 
            'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H', 'TX_RESP_Q003_I', 'TX_RESP_Q003_J', 
            'TX_RESP_Q003_L', 'TX_RESP_Q004_A', 'TX_RESP_Q004_B', 'TX_RESP_Q004_C', 'TX_RESP_Q004_D', 'CO_REGIAO',
            'TP_SITUACAO_FUNCIONAMENTO', 'TP_REGULAMENTACAO',
            'CO_MESORREGIAO', 'CO_MICRORREGIAO', 'CO_DISTRITO', 'TP_LOCALIZACAO',  'CO_ORGAO_REGIONAL', 'TP_DEPENDENCIA',
            'TX_RESP_Q004_E', 'TX_RESP_Q004_F', 'TX_RESP_Q004_G', 'TX_RESP_Q004_H', 'ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF', 'ID_MUNICIPIO', 
            'DT_ANO_LETIVO_INICIO', 'DT_ANO_LETIVO_TERMINO', 'IN_SITUACAO_CENSO', 'NU_ANO_CENSO', 'CO_UF', 'CO_MUNICIPIO', 'Ideb-2017']'''

'''delete = ['CO_MESORREGIAO', 'CO_MICRORREGIAO', 'CO_DISTRITO', 'TP_LOCALIZACAO',  'CO_ORGAO_REGIONAL', 'TP_DEPENDENCIA',
          'ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF', 'ID_MUNICIPIO', 'IN_SITUACAO_CENSO', 'Ideb-2017']'''


#X = dataset.drop(delete, axis=1)

'''X = dataset.drop(['Ideb-2017','IN_SITUACAO_CENSO', 'TX_RESP_Q001_A', 'TX_RESP_Q001_B', 'TX_RESP_Q002_A', 'TX_RESP_Q002_B', 'TX_RESP_Q002_C', 'TX_RESP_Q002_D', 
            'TX_RESP_Q002_E', 'TX_RESP_Q002_F', 'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D', 
            'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H', 'TX_RESP_Q003_I', 'TX_RESP_Q003_J', 
            'TX_RESP_Q003_L', 'TX_RESP_Q004_A', 'TX_RESP_Q004_B', 'TX_RESP_Q004_C', 'TX_RESP_Q004_D',], axis=1)'''

X = dataset.drop(['Ideb-2017','IN_SITUACAO_CENSO'], axis=1)

In [ ]:
list(X.columns)

['ID_PROVA_BRASIL',
 'ID_REGIAO',
 'ID_UF',
 'ID_MUNICIPIO',
 'ID_AREA',
 'ID_DEPENDENCIA_ADM',
 'ID_LOCALIZACAO',
 'TX_RESP_Q001_A',
 'TX_RESP_Q001_B',
 'TX_RESP_Q020_A',
 'TX_RESP_Q020_B',
 'TX_RESP_Q021_A',
 'TX_RESP_Q021_B',
 'TX_RESP_Q024_A',
 'TX_RESP_Q024_B',
 'TX_RESP_Q025_A',
 'TX_RESP_Q025_B',
 'TX_RESP_Q027_A',
 'TX_RESP_Q027_B',
 'TX_RESP_Q028_A',
 'TX_RESP_Q028_B',
 'TX_RESP_Q029_A',
 'TX_RESP_Q029_B',
 'TX_RESP_Q030_A',
 'TX_RESP_Q030_B',
 'TX_RESP_Q031_A',
 'TX_RESP_Q031_B',
 'TX_RESP_Q038_A',
 'TX_RESP_Q038_B',
 'TX_RESP_Q043_A',
 'TX_RESP_Q043_B',
 'TX_RESP_Q044_A',
 'TX_RESP_Q044_B',
 'TX_RESP_Q052_A',
 'TX_RESP_Q052_B',
 'TX_RESP_Q018_A',
 'TX_RESP_Q018_B',
 'TX_RESP_Q018_C',
 'TX_RESP_Q022_A',
 'TX_RESP_Q022_B',
 'TX_RESP_Q022_C',
 'TX_RESP_Q026_A',
 'TX_RESP_Q026_B',
 'TX_RESP_Q026_C',
 'TX_RESP_Q032_A',
 'TX_RESP_Q032_B',
 'TX_RESP_Q032_C',
 'TX_RESP_Q033_A',
 'TX_RESP_Q033_B',
 'TX_RESP_Q033_C',
 'TX_RESP_Q034_A',
 'TX_RESP_Q034_B',
 'TX_RESP_Q034_C',
 'TX_RESP_Q

In [ ]:
X

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,TX_RESP_Q022_B,...,IN_COMUM_CRECHE,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentesDivSocCulta,QtdeDocentesAtendEsp,QtdeDocentesTotal,QtdeTurmasProf,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,asQtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdedocentesCLT,QtdeDocentesComLicenciatura,QtdeAlunosTotal
29211590,2017,2,29,2900108,2,2,1,36,36,65,7,58,14,63,6,47,20,68,0,65,3,62,6,66,1,49,19,22,44,21,43,44,23,46,21,63,7,2,49,16,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,105,0,0,0,3,0,0,116,116,116,0,0,110,6,0,0,105,287
29032539,2017,2,29,2900207,2,2,1,52,95,124,23,107,39,95,53,72,73,145,2,137,9,133,13,142,4,110,36,35,107,87,50,121,22,74,68,110,29,6,78,64,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,95,0,0,37,3,0,0,147,147,147,0,0,95,52,0,0,127,685
29033047,2017,2,29,2900207,2,2,2,48,49,81,21,68,33,67,36,42,57,99,4,95,7,87,15,100,2,78,23,23,79,36,62,76,24,64,35,77,20,4,55,42,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,96,0,0,6,5,0,0,108,108,108,0,0,69,39,0,0,102,442
29446740,2017,2,29,2900306,2,2,1,34,65,90,11,82,19,76,25,54,45,100,1,94,6,83,16,96,3,70,28,29,71,27,68,82,14,58,40,82,13,4,60,39,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,74,12,0,78,0,0,0,154,154,154,0,0,69,85,0,0,129,689
29236606,2017,2,29,2900504,2,2,1,33,64,85,17,76,26,66,34,47,53,100,2,88,13,82,19,99,3,79,23,16,84,26,74,74,27,48,54,86,11,4,68,30,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,64,0,0,28,0,0,0,130,130,130,0,0,57,73,0,0,104,551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,2017,2,29,2933307,2,2,1,25,34,52,10,41,20,45,16,33,27,61,1,52,9,53,9,60,1,39,22,19,40,5,54,48,11,43,17,36,21,4,39,21,...,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,167,4,0,10,12,0,0,182,182,182,0,0,167,15,0,0,170,877
29433410,2017,2,29,2933307,2,2,1,54,54,103,8,83,27,103,9,71,42,111,3,99,15,88,26,111,2,69,45,44,69,20,91,86,28,55,57,93,16,3,63,46,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,208,14,4,6,7,0,0,214,214,214,0,0,214,0,0,0,210,822
29456487,2017,2,29,2933307,2,2,2,36,61,79,18,68,28,64,28,48,44,91,3,82,12,79,15,90,4,54,40,14,78,30,60,61,31,53,37,80,11,2,64,28,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,125,7,0,19,4,0,0,151,151,151,0,0,99,52,0,0,138,370
29457491,2017,2,29,2933307,2,2,2,27,33,39,16,35,20,37,19,27,31,55,2,47,10,48,9,54,3,40,14,15,41,21,34,47,11,32,22,47,12,0,34,24,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,9,0,0,0,0,0,68,68,68,0,0,60,8,0,0,68,268


In [ ]:
filteredColumns = X.dtypes[X.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

#X[listOfColumnNames].fillna(-1, inplace=True)

#X[listOfColumnNames] = X[listOfColumnNames].astype(int)

X[listOfColumnNames].isnull().sum()

Series([], dtype: float64)

In [ ]:
X_transform = minmax(X)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 68.35 %
Naive Bayes: Acurácia do modelo: 64.56 %
Decision Tree: Acurácia do modelo: 59.49 %
SVM: Acurácia do modelo: 63.29 %
Logistic Regression: Acurácia do modelo: 67.09 %
KNN: Acurácia do modelo: 59.49 %


**-Segunda Análise: Aplicando Seleção de Atributos (Embutida)**

In [ ]:
X_embedded = embedded_based(X, X_transform, y, 100)

X_embedded

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


,ID_DEPENDENCIA_ADM,TX_RESP_Q020_B,TX_RESP_Q024_B,TX_RESP_Q029_B,TX_RESP_Q043_A,TX_RESP_Q018_B,TX_RESP_Q022_B,TX_RESP_Q022_C,TX_RESP_Q026_A,TX_RESP_Q026_C,TX_RESP_Q032_C,TX_RESP_Q040_B,TX_RESP_Q040_C,TX_RESP_Q041_B,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_B,TX_RESP_Q055_A,TX_RESP_Q055_B,TX_RESP_Q057_B,TX_RESP_Q058_B,TX_RESP_Q039_A,TX_RESP_Q039_C,TX_RESP_Q045_B,TX_RESP_Q053_B,TX_RESP_Q053_C,TX_RESP_Q060_A,TX_RESP_Q060_D,TX_RESP_Q005_D,TX_RESP_Q007_D,TX_RESP_Q007_E,TX_RESP_Q009_D,TX_RESP_Q010_E,TX_RESP_Q011_D,TX_RESP_Q012_B,TX_RESP_Q012_C,TX_RESP_Q012_D,TX_RESP_Q012_E,TX_RESP_Q013_E,TX_RESP_Q014_A,...,TP_DEPENDENCIA,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_OUTROS,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,IN_AGUA_CACIMBA,IN_ESGOTO_REDE_PUBLICA,IN_ESGOTO_FOSSA,IN_LIXO_QUEIMA,IN_LIXO_OUTROS,IN_SALA_DIRETORIA,IN_SALA_PROFESSOR,IN_SALA_ATENDIMENTO_ESPECIAL,IN_QUADRA_ESPORTES_COBERTA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_EI,IN_BANHEIRO_PNE,IN_SECRETARIA,IN_BANHEIRO_CHUVEIRO,IN_PATIO_COBERTO,IN_PATIO_DESCOBERTO,IN_LAVANDERIA,IN_EQUIP_PARABOLICA,IN_EQUIP_COPIADORA,IN_EQUIP_RETROPROJETOR,IN_EQUIP_MULTIMIDIA,IN_EQUIP_FOTO,NU_EQUIP_FOTO,IN_FUNDAMENTAL_CICLOS,IN_MATERIAL_ESP_QUILOMBOLA,IN_MATERIAL_ESP_INDIGENA,IN_MATERIAL_ESP_NAO_UTILIZA,IN_FINAL_SEMANA,IN_EJA,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_EJA_PROF,IN_COMUM_PROF,QtdeDocentesTempo,QtdeAlunosTotal
29211590,2,7,6,6,21,7,16,3,39,24,12,1,0,5,1,10,48,42,17,51,16,11,1,2,4,15,7,8,4,1,0,0,0,0,26,6,3,0,0,0,...,2,1,1,0,1,1,0,0,1,0,0,1,1,0,0,0,0,0,1,0,1,0,0,1,1,1,1,1,1,0,0,0,1,0,0,0,0,0,6,287
29032539,2,23,53,13,87,29,64,5,86,47,23,5,0,16,10,11,82,69,42,93,26,31,4,5,4,43,29,4,3,1,1,1,1,0,25,3,0,0,0,6,...,2,1,1,0,1,1,0,1,0,0,0,1,1,0,0,0,0,1,1,1,1,0,0,1,1,1,1,1,2,0,0,0,1,0,0,0,0,0,52,685
29033047,2,21,36,15,36,20,42,6,65,28,32,0,3,17,6,5,53,47,33,69,30,37,2,1,0,29,15,3,1,0,0,0,0,0,19,1,0,0,0,3,...,2,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,39,442
29446740,2,11,25,16,27,13,39,2,67,29,22,8,3,13,5,3,67,80,10,67,13,39,3,1,8,40,16,1,4,2,1,0,1,0,20,3,0,1,1,1,...,2,1,1,0,1,1,0,0,1,0,0,1,1,0,0,0,0,1,1,1,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,85,689
29236606,2,17,34,19,26,11,30,4,46,44,27,3,0,10,2,7,74,70,22,78,36,24,2,4,2,58,14,1,1,0,0,0,0,0,32,4,1,0,0,1,...,2,1,1,0,1,1,0,0,1,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,73,551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,2,10,16,9,5,21,21,2,21,25,12,6,2,7,5,10,39,39,16,42,13,22,1,2,4,14,5,8,2,0,0,0,0,0,24,3,0,1,0,0,...,2,1,2,0,1,1,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0,1,0,0,0,15,877
29433410,2,8,9,26,20,16,46,5,33,43,28,13,2,15,3,10,85,63,38,91,14,36,2,0,4,34,13,6,17,1,0,0,0,0,40,2,2,0,0,0,...,2,1,1,0,1,1,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,822
29456487,2,18,28,15,30,11,28,3,50,26,17,2,0,18,3,6,69,51,28,74,36,3,1,7,4,26,3,45,5,0,2,0,0,0,22,5,1,0,0,0,...,2,1,1,0,1,1,0,1,0,0,0,1,1,0,0,0,0,1,1,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,52,370
29457491,2,16,19,9,21,12,24,1,31,10,10,0,0,12,7,9,33,35,14,38,27,2,3,0,3,17,1,49,1,0,0,0,0,0,6,0,0,0,0,2,...,2,1,2,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,8,268


In [ ]:
X_transform_em = minmax(X_embedded)

X_train, X_test, y_train, y_test = train_test_split(X_transform_em, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 68.35 %
Naive Bayes: Acurácia do modelo: 64.56 %
Decision Tree: Acurácia do modelo: 51.9 %
SVM: Acurácia do modelo: 68.35 %
Logistic Regression: Acurácia do modelo: 68.35 %
KNN: Acurácia do modelo: 56.96 %


**-Segunda Análise: Aplicando Seleção de Atributos (Wrapper)**

In [ ]:
X_wrapper = wrapper_based(X, X_transform, y, 100)

X_wrapper

,ID_DEPENDENCIA_ADM,TX_RESP_Q001_A,TX_RESP_Q020_B,TX_RESP_Q024_B,TX_RESP_Q030_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q018_B,TX_RESP_Q022_B,TX_RESP_Q022_C,TX_RESP_Q026_C,TX_RESP_Q032_C,TX_RESP_Q040_C,TX_RESP_Q041_B,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q046_B,TX_RESP_Q047_A,TX_RESP_Q047_B,TX_RESP_Q048_B,TX_RESP_Q055_A,TX_RESP_Q055_B,TX_RESP_Q056_A,TX_RESP_Q057_B,TX_RESP_Q058_B,TX_RESP_Q039_A,TX_RESP_Q039_C,TX_RESP_Q045_B,TX_RESP_Q053_B,TX_RESP_Q053_C,TX_RESP_Q060_A,TX_RESP_Q060_D,TX_RESP_Q005_C,TX_RESP_Q005_D,TX_RESP_Q007_D,TX_RESP_Q007_E,TX_RESP_Q008_E,TX_RESP_Q009_D,TX_RESP_Q010_C,...,TX_RESP_Q019_A,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_C,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_H,TX_RESP_Q003_G,TX_RESP_Q003_H,CO_ORGAO_REGIONAL,TP_DEPENDENCIA,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_OUTROS,IN_AGUA_REDE_PUBLICA,IN_AGUA_CACIMBA,IN_AGUA_INEXISTENTE,IN_LIXO_QUEIMA,IN_LIXO_OUTROS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_QUADRA_ESPORTES_COBERTA,IN_BANHEIRO_EI,IN_BANHEIRO_CHUVEIRO,IN_ALOJAM_PROFESSOR,IN_LAVANDERIA,IN_DEPENDENCIAS_OUTRAS,IN_EQUIP_COPIADORA,NU_EQUIP_TV,NU_EQUIP_DVD,NU_EQUIP_FOTO,TP_AEE,IN_FUNDAMENTAL_CICLOS,IN_MATERIAL_ESP_QUILOMBOLA,IN_MATERIAL_ESP_INDIGENA,IN_MATERIAL_ESP_NAO_UTILIZA,IN_EJA,IN_COMUM_PRE,IN_COMUM_MEDIO_INTEGRADO,asQtdeDocentesEfetivos
29211590,2,36,7,6,1,21,43,7,16,3,24,12,0,5,1,10,8,48,20,36,8,42,17,41,51,16,11,1,2,4,15,7,8,20,4,1,0,0,0,0,...,6,4,7,8,10,10,0,8,5,3,3,2,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,16,2,1,0,0,0,0,1,0,0,0,110
29032539,2,52,23,53,4,87,50,29,64,5,47,23,0,16,10,11,28,82,31,77,30,69,42,71,93,26,31,4,5,4,43,29,4,16,3,1,1,0,1,3,...,9,9,29,25,12,45,0,12,15,10,24,2,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,16,4,2,0,0,0,0,1,0,0,0,95
29033047,2,48,21,36,2,36,62,20,42,6,28,32,3,17,6,5,22,53,29,46,18,47,33,57,69,30,37,2,1,0,29,15,3,4,1,0,0,0,0,0,...,11,5,24,14,12,30,1,7,6,12,24,2,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,9,2,0,0,0,0,0,1,0,0,0,69
29446740,2,34,11,25,3,27,68,13,39,2,29,22,3,13,5,3,13,67,19,59,8,80,10,44,67,13,39,3,1,8,40,16,1,14,4,2,1,1,0,0,...,1,9,12,8,17,25,0,3,10,10,18,2,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,69
29236606,2,33,17,34,3,26,74,11,30,4,44,27,0,10,2,7,13,74,26,60,15,70,22,40,78,36,24,2,4,2,58,14,1,15,1,0,0,0,0,0,...,8,4,20,14,14,34,0,2,8,10,12,2,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,1,1,0,0,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,2,25,10,16,1,5,54,21,21,2,25,12,2,7,5,10,8,39,19,30,5,39,16,29,42,13,22,1,2,4,14,5,8,23,2,0,0,0,0,0,...,3,4,9,6,10,15,0,7,1,6,20,2,1,2,0,1,0,0,0,0,0,0,0,0,0,0,0,1,14,2,0,0,0,0,0,1,1,0,0,167
29433410,2,54,8,9,2,20,91,16,46,5,43,28,2,15,3,10,14,85,25,73,18,63,38,45,91,14,36,2,0,4,34,13,6,36,17,1,0,0,0,2,...,4,8,12,4,22,21,0,6,16,5,20,2,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,12,0,0,0,0,0,0,1,1,0,0,214
29456487,2,36,18,28,4,30,60,11,28,3,26,17,0,18,3,6,7,69,19,56,14,51,28,39,74,36,3,1,7,4,26,3,45,21,5,0,2,0,0,1,...,9,1,11,15,5,22,0,8,5,8,20,2,1,1,0,1,0,0,0,0,0,0,0,1,1,0,0,1,5,2,0,0,0,0,0,1,0,0,0,99
29457491,2,27,16,19,3,21,34,12,24,1,10,10,0,12,7,9,15,33,15,33,4,35,14,17,38,27,2,3,0,3,17,1,49,11,1,0,0,0,0,1,...,12,1,3,16,5,11,0,14,6,3,20,2,1,2,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,60


In [ ]:
X_transform_w = minmax(X_wrapper)

X_train, X_test, y_train, y_test = train_test_split(X_transform_w, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 65.82 %
Naive Bayes: Acurácia do modelo: 68.35 %
Decision Tree: Acurácia do modelo: 54.43 %
SVM: Acurácia do modelo: 69.62 %
Logistic Regression: Acurácia do modelo: 70.89 %
KNN: Acurácia do modelo: 60.76 %


**-Segunda Análise: Aplicando Seleção de Atributos (Filtragem)**

In [ ]:
X_selection = selecao_atributos(X, y, 100)

X_selection

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [  0   1   2 234 236 239 242 247 248 249 250 251 252 253 254 258 260 261
 262 263 264 265 278 280 303 355 363 364 365 369 370 372 373 382 383 384
 385 386 387 388 389 393 394 395 397 403 404] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


,ID_DEPENDENCIA_ADM,TX_RESP_Q020_B,TX_RESP_Q024_B,TX_RESP_Q043_A,TX_RESP_Q018_B,TX_RESP_Q035_C,TX_RESP_Q040_C,TX_RESP_Q041_B,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q055_A,TX_RESP_Q039_B,TX_RESP_Q039_C,TX_RESP_Q045_B,TX_RESP_Q060_D,TX_RESP_Q005_A,TX_RESP_Q005_C,TX_RESP_Q005_D,TX_RESP_Q005_E,TX_RESP_Q008_A,TX_RESP_Q009_C,TX_RESP_Q011_D,TX_RESP_Q012_A,TX_RESP_Q012_B,TX_RESP_Q012_C,TX_RESP_Q013_A,TX_RESP_Q013_B,TX_RESP_Q013_C,TX_RESP_Q013_D,TX_RESP_Q013_E,TX_RESP_Q014_A,TX_RESP_Q014_C,TX_RESP_Q014_D,TX_RESP_Q014_E,TX_RESP_Q015_B,TX_RESP_Q036_A,TX_RESP_Q036_C,TX_RESP_Q036_E,TX_RESP_Q037_B,TX_RESP_Q002_A,...,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_PNE,IN_BANHEIRO_CHUVEIRO,IN_REFEITORIO,IN_AUDITORIO,IN_PATIO_COBERTO,IN_PATIO_DESCOBERTO,IN_AREA_VERDE,NU_SALAS_EXISTENTES,IN_EQUIP_PARABOLICA,IN_EQUIP_RETROPROJETOR,IN_EQUIP_SOM,IN_EQUIP_FAX,IN_EQUIP_FOTO,NU_EQUIP_VIDEOCASSETE,NU_EQUIP_DVD,NU_EQUIP_PARABOLICA,NU_EQUIP_RETROPROJETOR,NU_EQUIP_IMPRESSORA,NU_EQUIP_IMPRESSORA_MULT,NU_EQUIP_SOM,NU_EQUIP_MULTIMIDIA,NU_EQUIP_FAX,NU_EQUIP_FOTO,NU_COMPUTADOR,NU_COMP_ADMINISTRATIVO,NU_COMP_ALUNO,NU_FUNCIONARIOS,IN_EJA,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_PROF,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesTotal,QtdeTurmasProf,QtdeDocentes,asQtdeDocentesEfetivos
29211590,2,7,6,21,7,0,0,5,1,10,42,11,1,2,8,3,20,4,3,3,1,0,36,26,6,36,33,2,1,0,0,15,1,0,1,12,2,13,6,30,...,1,1,0,0,0,0,1,0,0,16,1,1,1,1,1,0,2,1,1,0,0,3,1,2,1,13,2,5,36,0,1,0,0,105,0,0,116,116,116,110
29032539,2,23,53,87,29,5,0,16,10,11,69,25,4,5,4,11,16,3,1,19,1,0,117,25,3,117,23,7,1,0,6,22,4,1,8,43,13,16,36,14,...,1,1,1,1,1,0,1,0,0,10,1,1,1,0,1,0,4,1,1,6,4,5,3,0,2,9,2,0,40,0,1,0,0,95,0,0,147,147,147,95
29033047,2,21,36,36,20,9,3,17,6,5,47,22,2,1,3,8,4,1,1,3,1,0,78,19,1,81,21,0,0,0,3,8,1,2,6,33,12,14,17,7,...,1,1,1,0,0,0,0,0,0,6,0,1,1,1,0,0,2,0,1,2,0,1,0,1,0,14,3,11,26,0,1,0,0,96,0,0,108,108,108,69
29446740,2,11,25,27,13,5,3,13,5,3,80,10,3,1,1,7,14,4,1,6,1,0,77,20,3,66,28,6,0,1,1,24,4,0,5,18,9,19,9,8,...,1,1,1,1,0,0,0,1,0,7,0,1,0,0,0,0,0,0,1,3,1,0,1,0,0,6,3,3,36,1,1,0,0,74,12,0,154,154,154,69
29236606,2,17,34,26,11,8,0,10,2,7,70,11,2,4,1,8,15,1,1,1,0,0,63,32,4,74,27,1,0,0,1,16,1,0,0,25,6,18,22,9,...,0,1,1,0,0,0,0,0,0,8,0,1,1,0,0,0,1,0,1,5,0,1,0,0,0,17,3,10,44,1,1,0,0,64,0,0,130,130,130,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,2,10,16,5,21,2,2,7,5,10,39,8,1,2,8,0,23,2,0,1,1,0,33,24,3,29,30,2,1,0,0,13,3,0,4,11,3,10,10,9,...,0,0,1,0,0,0,0,0,0,8,1,1,1,1,0,0,2,1,2,5,0,6,0,1,0,15,5,10,39,1,1,0,0,167,4,0,182,182,182,167
29433410,2,8,9,20,16,2,2,15,3,10,63,17,2,0,6,1,36,17,3,0,2,0,68,40,2,33,64,11,2,0,0,30,4,1,0,13,5,18,12,21,...,1,1,0,0,0,1,1,0,0,8,0,0,0,1,0,0,0,0,0,5,0,0,0,1,0,15,3,10,45,1,1,0,0,208,14,4,214,214,214,214
29456487,2,18,28,30,11,3,0,18,3,6,51,64,1,7,45,5,21,5,2,1,2,0,67,22,5,64,30,2,0,0,0,16,3,0,1,23,7,14,23,17,...,0,0,1,1,0,0,1,0,0,6,0,0,1,0,0,0,2,0,0,1,0,1,1,0,0,5,5,0,102,0,1,0,0,125,7,0,151,151,151,99
29457491,2,16,19,21,12,4,0,12,7,9,35,35,3,0,49,3,11,1,0,5,1,0,53,6,0,46,10,1,1,0,2,5,0,0,3,15,6,6,15,1,...,0,0,0,0,0,0,0,0,0,8,0,0,1,0,0,0,1,0,0,1,0,1,1,0,0,2,2,0,20,0,1,0,0,68,9,0,68,68,68,60


In [ ]:
X_transform = minmax(X_selection)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_transform)
naive_result = naive.predict(X_transform)
decision_result = decision_tree.predict(X_transform)
svm_result = svm.predict(X_transform)
logistic_result = r_logistic.predict(X_transform)
knn_result = knn.predict(X_transform)

print("Random forest: " + acuracia(random_result, y))
print("Naive Bayes: " + acuracia(naive_result, y))
print("Decision Tree: " + acuracia(decision_result, y))
print("SVM: " + acuracia(svm_result, y))
print("Logistic Regression: "+acuracia(logistic_result, y))
print("KNN: "+acuracia(knn_result, y))

Random forest: Acurácia do modelo: 93.38 %
Naive Bayes: Acurácia do modelo: 63.36 %
Decision Tree: Acurácia do modelo: 91.35 %
SVM: Acurácia do modelo: 74.3 %
Logistic Regression: Acurácia do modelo: 74.55 %
KNN: Acurácia do modelo: 70.74 %


**Segunda Análise: Unindo subconjunto de melhores atributos**

In [ ]:
X_all = pd.concat([X_selection, X_wrapper], axis=1)

X_all = pd.concat([X_all, X_embedded], axis=1)

In [ ]:
#Removendo colunas duplicadas
X_new_all = X_all.T.drop_duplicates().T

In [ ]:
X_transform = minmax(X_new_all)
#X_transform = minmax(X_all)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_transform)
naive_result = naive.predict(X_transform)
decision_result = decision_tree.predict(X_transform)
svm_result = svm.predict(X_transform)
logistic_result = r_logistic.predict(X_transform)
knn_result = knn.predict(X_transform)

print("Random forest: " + acuracia(random_result, y))
print("Naive Bayes: " + acuracia(naive_result, y))
print("Decision Tree: " + acuracia(decision_result, y))
print("SVM: " + acuracia(svm_result, y))
print("Logistic Regression: "+acuracia(logistic_result, y))
print("KNN: "+acuracia(knn_result, y))

Random forest: Acurácia do modelo: 93.38 %
Naive Bayes: Acurácia do modelo: 64.89 %
Decision Tree: Acurácia do modelo: 92.37 %
SVM: Acurácia do modelo: 82.44 %
Logistic Regression: Acurácia do modelo: 82.19 %
KNN: Acurácia do modelo: 71.76 %


Resultado com Todos atributos

- Random forest: Acurácia do modelo: 93.64 %
- Naive Bayes: Acurácia do modelo: 64.89 %
- Decision Tree: Acurácia do modelo: 91.35 %
- SVM: Acurácia do modelo: 83.97 %
- Logistic Regression: Acurácia do modelo: 83.46 %
- KNN: Acurácia do modelo: 74.05 %

**Segunda Análise: Aplicando a técnica merge**

Link da Dissertação: https://www.dcc.ufmg.br/pos/cursos/defesas/1930M.PDF


In [ ]:
#Metodo Merge combina os atributos mais frequentes nos melhores conjuntos de seleção 
#Cada atributo irá apresentar um score de acordo com sua presença em um subconjunto de dados
def metodo_merge(selection, embedded, wrapper, ranking):
  for atributo in selection:
    score = ranking[atributo]
    if (atributo in embedded):
      score += 1
      embedded.remove(atributo)
    if (atributo in wrapper):
      score +=1
      wrapper.remove(atributo)
    ranking[atributo] = score

  for atributo in embedded:
    score = ranking[atributo]
    if (atributo in selection):
      score += 1
      embedded.remove(atributo)
    if (atributo in wrapper):
      score +=1
      wrapper.remove(atributo)
    ranking[atributo] = score

  for atributo in wrapper:
    score = ranking[atributo]
    if (atributo in selection):
      score += 1
      embedded.remove(atributo)
    if (atributo in embedded):
      score +=1
      wrapper.remove(atributo)
    ranking[atributo] = score
 
  return ranking      

In [ ]:
#Gerando Listas com os melhores atributos de cada abordagem
selection = list(X_selection.columns)
embedded = list(X_embedded.columns)
wrapper = list(X_wrapper.columns)

#Criando dicionário
values = [1]*300

all = selection + embedded + wrapper

ranking = dict(zip(all, values))

ranking = metodo_merge(selection, embedded, wrapper, ranking)

#Filtrando os atributos que estão em mais de um subconjunto

dic_aux = ranking.items()

filtered = {key:value for key, value in dic_aux if value > 1}

#Gerando X após o merge

X_merge = dataset[list(filtered.keys())]

In [ ]:
X_transform = minmax(X_merge)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_transform)
naive_result = naive.predict(X_transform)
decision_result = decision_tree.predict(X_transform)
svm_result = svm.predict(X_transform)
logistic_result = r_logistic.predict(X_transform)
knn_result = knn.predict(X_transform)

print("Random forest: " + acuracia(random_result, y))
print("Naive Bayes: " + acuracia(naive_result, y))
print("Decision Tree: " + acuracia(decision_result, y))
print("SVM: " + acuracia(svm_result, y))
print("Logistic Regression: "+acuracia(logistic_result, y))
print("KNN: "+acuracia(knn_result, y))

Random forest: Acurácia do modelo: 93.89 %
Naive Bayes: Acurácia do modelo: 67.43 %
Decision Tree: Acurácia do modelo: 90.84 %
SVM: Acurácia do modelo: 79.9 %
Logistic Regression: Acurácia do modelo: 78.12 %
KNN: Acurácia do modelo: 72.77 %


In [ ]:
dados_para_exportar = pd.DataFrame(columns=X_merge.columns, data=X_merge.values, index=X_merge.index)
dados_para_exportar.insert(len(dados_para_exportar.columns), "Valor Real Ideb", y.to_numpy())
dados_para_exportar.insert(len(dados_para_exportar.columns), "Previsão Ideb", random_result)


dados_para_exportar

,ID_DEPENDENCIA_ADM,TX_RESP_Q020_B,TX_RESP_Q024_B,TX_RESP_Q043_A,TX_RESP_Q018_B,TX_RESP_Q040_C,TX_RESP_Q041_B,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q055_A,TX_RESP_Q039_C,TX_RESP_Q045_B,TX_RESP_Q060_D,TX_RESP_Q005_C,TX_RESP_Q005_D,TX_RESP_Q011_D,TX_RESP_Q012_B,TX_RESP_Q012_C,TX_RESP_Q013_D,TX_RESP_Q013_E,TX_RESP_Q014_A,TX_RESP_Q014_C,TX_RESP_Q014_E,TX_RESP_Q015_B,TX_RESP_Q002_A,TX_RESP_Q016_A,TX_RESP_Q019_A,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_G,TX_RESP_Q004_H,TP_DEPENDENCIA,TP_OCUPACAO_PREDIO_ESCOLAR,IN_AGUA_CACIMBA,IN_SALA_ATENDIMENTO_ESPECIAL,IN_BANHEIRO_PNE,IN_BANHEIRO_CHUVEIRO,IN_PATIO_COBERTO,IN_PATIO_DESCOBERTO,...,asQtdeDocentesEfetivos,TX_RESP_Q022_B,TX_RESP_Q022_C,TX_RESP_Q026_C,TX_RESP_Q032_C,TX_RESP_Q046_B,TX_RESP_Q055_B,TX_RESP_Q057_B,TX_RESP_Q058_B,TX_RESP_Q039_A,TX_RESP_Q053_B,TX_RESP_Q053_C,TX_RESP_Q060_A,TX_RESP_Q007_D,TX_RESP_Q007_E,TX_RESP_Q009_D,TX_RESP_Q010_E,TX_RESP_Q012_D,TX_RESP_Q012_E,TX_RESP_Q015_E,TX_RESP_Q017_E,TX_RESP_Q002_B,TX_RESP_Q016_B,TX_RESP_Q023_C,CO_ORGAO_REGIONAL,TP_REGULAMENTACAO,IN_LOCAL_FUNC_OUTROS,IN_AGUA_REDE_PUBLICA,IN_LIXO_QUEIMA,IN_LIXO_OUTROS,IN_QUADRA_ESPORTES_COBERTA,IN_BANHEIRO_EI,IN_LAVANDERIA,IN_EQUIP_COPIADORA,IN_FUNDAMENTAL_CICLOS,IN_MATERIAL_ESP_QUILOMBOLA,IN_MATERIAL_ESP_INDIGENA,IN_MATERIAL_ESP_NAO_UTILIZA,Valor Real Ideb,Previsão Ideb
29211590,2,7,6,21,7,0,5,1,10,42,1,2,8,20,4,0,26,6,1,0,0,15,0,1,30,1,6,4,7,8,10,8,2,1,0,0,0,0,1,0,...,110,16,3,24,12,48,17,51,16,11,4,15,7,1,0,0,0,3,0,16,0,6,8,10,3,1,0,1,0,0,0,0,0,1,0,0,0,1,acima da média,acima da média
29032539,2,23,53,87,29,0,16,10,11,69,4,5,4,16,3,0,25,3,1,0,6,22,1,8,14,1,9,9,29,25,45,12,2,1,0,0,1,1,1,0,...,95,64,5,47,23,82,42,93,26,31,4,43,29,1,1,1,1,0,0,19,0,30,15,12,24,1,0,1,0,0,0,0,0,1,0,0,0,1,acima da média,abaixo da média
29033047,2,21,36,36,20,3,17,6,5,47,2,1,3,4,1,0,19,1,0,0,3,8,2,6,7,2,11,5,24,14,30,7,2,1,1,0,1,0,0,0,...,69,42,6,28,32,53,33,69,30,37,0,29,15,0,0,0,0,0,0,10,0,15,17,12,24,1,0,1,0,0,0,0,0,1,0,0,0,1,média,abaixo da média
29446740,2,11,25,27,13,3,13,5,3,80,3,1,1,14,4,0,20,3,0,1,1,24,0,5,8,0,1,9,12,8,25,3,2,1,0,0,1,1,0,1,...,69,39,2,29,22,67,10,67,13,39,8,40,16,2,1,0,1,0,1,10,1,17,9,17,18,1,0,1,0,0,0,0,0,0,0,0,0,1,abaixo da média,abaixo da média
29236606,2,17,34,26,11,0,10,2,7,70,2,4,1,15,1,0,32,4,0,0,1,16,0,0,9,0,8,4,20,14,34,2,2,1,0,0,1,0,0,0,...,57,30,4,44,27,74,22,78,36,24,2,58,14,0,0,0,0,1,0,41,1,23,5,14,12,1,0,1,0,0,0,0,0,0,0,0,0,1,média,média
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,2,10,16,5,21,2,7,5,10,39,1,2,8,23,2,0,24,3,1,0,0,13,0,4,9,1,3,4,9,6,15,7,2,2,0,0,1,0,0,0,...,167,21,2,25,12,39,16,42,13,22,4,14,5,0,0,0,0,0,1,6,0,9,12,10,20,1,0,1,0,0,0,0,0,1,0,0,0,1,abaixo da média,abaixo da média
29433410,2,8,9,20,16,2,15,3,10,63,2,0,6,36,17,0,40,2,2,0,0,30,1,0,21,1,4,8,12,4,21,6,2,1,0,0,0,0,1,0,...,214,46,5,43,28,85,38,91,14,36,4,34,13,1,0,0,0,2,0,7,0,19,10,22,20,1,0,1,0,0,0,0,0,1,0,0,0,1,acima da média,acima da média
29456487,2,18,28,30,11,0,18,3,6,51,1,7,45,21,5,0,22,5,0,0,0,16,0,1,17,1,9,1,11,15,22,8,2,1,0,0,1,1,1,0,...,99,28,3,26,17,69,28,74,36,3,4,26,3,0,2,0,0,1,0,23,0,13,6,5,20,1,0,1,0,0,0,0,0,1,0,0,0,1,acima da média,abaixo da média
29457491,2,16,19,21,12,0,12,7,9,35,3,0,49,11,1,0,6,0,1,0,2,5,0,3,1,1,12,1,3,16,11,14,2,2,1,0,0,0,0,0,...,60,24,1,10,10,33,14,38,27,2,3,17,1,0,0,0,0,0,0,7,0,12,3,5,20,1,0,0,1,0,0,0,0,1,0,0,0,1,média,média


In [ ]:
dados_para_exportar = dados_para_exportar.loc[dados_para_exportar['Valor Real Ideb'] ==  dados_para_exportar['Previsão Ideb']]


dados_para_exportar

,ID_DEPENDENCIA_ADM,TX_RESP_Q020_B,TX_RESP_Q024_B,TX_RESP_Q043_A,TX_RESP_Q018_B,TX_RESP_Q040_C,TX_RESP_Q041_B,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q055_A,TX_RESP_Q039_C,TX_RESP_Q045_B,TX_RESP_Q060_D,TX_RESP_Q005_C,TX_RESP_Q005_D,TX_RESP_Q011_D,TX_RESP_Q012_B,TX_RESP_Q012_C,TX_RESP_Q013_D,TX_RESP_Q013_E,TX_RESP_Q014_A,TX_RESP_Q014_C,TX_RESP_Q014_E,TX_RESP_Q015_B,TX_RESP_Q002_A,TX_RESP_Q016_A,TX_RESP_Q019_A,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_G,TX_RESP_Q004_H,TP_DEPENDENCIA,TP_OCUPACAO_PREDIO_ESCOLAR,IN_AGUA_CACIMBA,IN_SALA_ATENDIMENTO_ESPECIAL,IN_BANHEIRO_PNE,IN_BANHEIRO_CHUVEIRO,IN_PATIO_COBERTO,IN_PATIO_DESCOBERTO,...,asQtdeDocentesEfetivos,TX_RESP_Q022_B,TX_RESP_Q022_C,TX_RESP_Q026_C,TX_RESP_Q032_C,TX_RESP_Q046_B,TX_RESP_Q055_B,TX_RESP_Q057_B,TX_RESP_Q058_B,TX_RESP_Q039_A,TX_RESP_Q053_B,TX_RESP_Q053_C,TX_RESP_Q060_A,TX_RESP_Q007_D,TX_RESP_Q007_E,TX_RESP_Q009_D,TX_RESP_Q010_E,TX_RESP_Q012_D,TX_RESP_Q012_E,TX_RESP_Q015_E,TX_RESP_Q017_E,TX_RESP_Q002_B,TX_RESP_Q016_B,TX_RESP_Q023_C,CO_ORGAO_REGIONAL,TP_REGULAMENTACAO,IN_LOCAL_FUNC_OUTROS,IN_AGUA_REDE_PUBLICA,IN_LIXO_QUEIMA,IN_LIXO_OUTROS,IN_QUADRA_ESPORTES_COBERTA,IN_BANHEIRO_EI,IN_LAVANDERIA,IN_EQUIP_COPIADORA,IN_FUNDAMENTAL_CICLOS,IN_MATERIAL_ESP_QUILOMBOLA,IN_MATERIAL_ESP_INDIGENA,IN_MATERIAL_ESP_NAO_UTILIZA,Valor Real Ideb,Previsão Ideb
29211590,2,7,6,21,7,0,5,1,10,42,1,2,8,20,4,0,26,6,1,0,0,15,0,1,30,1,6,4,7,8,10,8,2,1,0,0,0,0,1,0,...,110,16,3,24,12,48,17,51,16,11,4,15,7,1,0,0,0,3,0,16,0,6,8,10,3,1,0,1,0,0,0,0,0,1,0,0,0,1,acima da média,acima da média
29446740,2,11,25,27,13,3,13,5,3,80,3,1,1,14,4,0,20,3,0,1,1,24,0,5,8,0,1,9,12,8,25,3,2,1,0,0,1,1,0,1,...,69,39,2,29,22,67,10,67,13,39,8,40,16,2,1,0,1,0,1,10,1,17,9,17,18,1,0,1,0,0,0,0,0,0,0,0,0,1,abaixo da média,abaixo da média
29236606,2,17,34,26,11,0,10,2,7,70,2,4,1,15,1,0,32,4,0,0,1,16,0,0,9,0,8,4,20,14,34,2,2,1,0,0,1,0,0,0,...,57,30,4,44,27,74,22,78,36,24,2,58,14,0,0,0,0,1,0,41,1,23,5,14,12,1,0,1,0,0,0,0,0,0,0,0,0,1,média,média
29425417,2,1,3,5,4,0,2,0,1,15,0,0,0,10,0,0,10,0,0,1,0,5,0,1,1,1,1,4,5,0,9,0,2,1,0,0,1,0,0,0,...,16,11,2,9,12,16,7,19,2,14,0,3,1,0,0,0,0,0,0,2,0,13,1,6,22,1,0,1,0,0,0,0,0,1,0,0,0,1,média,média
29150450,2,0,0,5,4,1,2,0,1,54,0,3,0,21,8,0,42,2,4,1,0,20,1,2,9,0,0,23,1,0,6,0,2,1,0,0,1,1,1,1,...,253,23,4,21,22,59,10,60,2,27,0,15,5,0,0,0,0,0,0,7,0,24,8,4,18,1,0,1,0,0,0,0,0,1,0,0,0,1,acima da média,acima da média
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29361540,1,0,4,2,15,0,2,0,3,65,0,4,0,23,10,0,32,8,9,3,1,25,3,4,21,1,1,29,1,1,4,1,1,1,0,1,0,1,1,0,...,296,29,1,22,25,57,9,63,2,34,2,18,14,1,0,0,0,0,0,11,0,9,15,16,0,1,0,1,0,0,1,0,0,1,0,0,0,1,acima da média,acima da média
29409500,2,10,16,5,21,2,7,5,10,39,1,2,8,23,2,0,24,3,1,0,0,13,0,4,9,1,3,4,9,6,15,7,2,2,0,0,1,0,0,0,...,167,21,2,25,12,39,16,42,13,22,4,14,5,0,0,0,0,0,1,6,0,9,12,10,20,1,0,1,0,0,0,0,0,1,0,0,0,1,abaixo da média,abaixo da média
29433410,2,8,9,20,16,2,15,3,10,63,2,0,6,36,17,0,40,2,2,0,0,30,1,0,21,1,4,8,12,4,21,6,2,1,0,0,0,0,1,0,...,214,46,5,43,28,85,38,91,14,36,4,34,13,1,0,0,0,2,0,7,0,19,10,22,20,1,0,1,0,0,0,0,0,1,0,0,0,1,acima da média,acima da média
29457491,2,16,19,21,12,0,12,7,9,35,3,0,49,11,1,0,6,0,1,0,2,5,0,3,1,1,12,1,3,16,11,14,2,2,1,0,0,0,0,0,...,60,24,1,10,10,33,14,38,27,2,3,17,1,0,0,0,0,0,0,7,0,12,3,5,20,1,0,0,1,0,0,0,0,1,0,0,0,1,média,média


In [ ]:
#dados_para_exportar.to_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/resultado_analise_merge_dataset_completo.csv')
#dados_para_exportar.to_csv('/content/drive/MyDrive/TCC/resultado_analise_merge_dataset_completo.csv')
dados_para_exportar.to_csv('/content/drive/MyDrive/TCC/resultado_analise_merge_dataset_completo_BA.csv')

#### -Terceira análise: variáveis das escolas e professores

In [ ]:
columns_escolas = dataset.columns[237:404]

X_escolas = dataset[columns_escolas]

X_escolas

,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_CATEGORIA_ESCOLA_PRIVADA,IN_CONVENIADA_PP,TP_CONVENIO_PODER_PUBLICO,IN_MANT_ESCOLA_PRIVADA_EMP,IN_MANT_ESCOLA_PRIVADA_ONG,IN_MANT_ESCOLA_PRIVADA_SIND,IN_MANT_ESCOLA_PRIVADA_SIST_S,IN_MANT_ESCOLA_PRIVADA_S_FINS,CO_ESCOLA_SEDE_VINCULADA,CO_IES_OFERTANTE,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,TP_OCUPACAO_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,IN_AGUA_POCO_ARTESIANO,IN_AGUA_CACIMBA,IN_AGUA_FONTE_RIO,...,IN_EDUCACAO_INDIGENA,TP_INDIGENA_LINGUA,CO_LINGUA_INDIGENA,IN_BRASIL_ALFABETIZADO,IN_FINAL_SEMANA,IN_FORMACAO_ALTERNANCIA,IN_MEDIACAO_PRESENCIAL,IN_MEDIACAO_SEMIPRESENCIAL,IN_MEDIACAO_EAD,IN_ESPECIAL_EXCLUSIVA,IN_REGULAR,IN_EJA,IN_PROFISSIONALIZANTE,IN_COMUM_CRECHE,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM
29211590,1,1496361600000000000,1513123200000000000,2,2906,29023,29,2900108,290010805,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,105,0,0,0,3
29032539,1,1496361600000000000,1513123200000000000,2,2902,29005,29,2900207,290020705,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,95,0,0,37,3
29033047,1,1496361600000000000,1513123200000000000,2,2902,29005,29,2900207,290020705,2,2,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,96,0,0,6,5
29446740,1,1496361600000000000,1513123200000000000,2,2904,29017,29,2900306,290030605,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,74,12,0,78,0
29236606,1,1496361600000000000,1513123200000000000,2,2906,29025,29,2900504,290050405,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,64,0,0,28,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,1,1496361600000000000,1513123200000000000,2,2906,29028,29,2933307,293330705,2,1,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,167,4,0,10,12
29433410,1,1496361600000000000,1513900800000000000,2,2906,29028,29,2933307,293330705,2,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,1,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,208,14,4,6,7
29456487,1,1496361600000000000,1513123200000000000,2,2906,29028,29,2933307,293330705,2,2,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,125,7,0,19,4
29457491,1,1496361600000000000,1513123200000000000,2,2906,29028,29,2933307,293330705,2,2,0,0,0,

In [ ]:
X_escolas.drop(columns=['DT_ANO_LETIVO_INICIO', 'DT_ANO_LETIVO_TERMINO'], axis=1, inplace=True)

X_transform = minmax(X_escolas)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 64.56 %
Naive Bayes: Acurácia do modelo: 60.76 %
Decision Tree: Acurácia do modelo: 54.43 %
SVM: Acurácia do modelo: 58.23 %
Logistic Regression: Acurácia do modelo: 63.29 %
KNN: Acurácia do modelo: 58.23 %


**Aplicando seleção de atributos (Filtragem)**

In [ ]:
X_selection = selecao_atributos(X_escolas, y, 100)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [  0   1   4   9  10  11  12  13  14  15  16  20  22  23  24  25  26  27
  40  42  65 117 125 126 127 131 132 134 135 144 145 146 147 148 149 150
 151 155 156 157 159] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 67.09 %
Naive Bayes: Acurácia do modelo: 60.76 %
Decision Tree: Acurácia do modelo: 49.37 %
SVM: Acurácia do modelo: 58.23 %
Logistic Regression: Acurácia do modelo: 63.29 %
KNN: Acurácia do modelo: 58.23 %


**Aplicando seleção de atributos (Embutida)**

In [ ]:
X_embedded = embedded_based(X, X_transform, y, 100)

X_embedded

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


,ID_UF,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q028_A,TX_RESP_Q029_A,TX_RESP_Q038_B,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q022_A,TX_RESP_Q026_C,TX_RESP_Q032_A,TX_RESP_Q033_A,TX_RESP_Q033_B,TX_RESP_Q033_C,TX_RESP_Q034_A,TX_RESP_Q034_B,TX_RESP_Q034_C,TX_RESP_Q035_A,TX_RESP_Q035_B,TX_RESP_Q040_A,TX_RESP_Q040_B,TX_RESP_Q040_C,TX_RESP_Q041_A,TX_RESP_Q041_B,TX_RESP_Q042_A,TX_RESP_Q046_B,TX_RESP_Q046_C,TX_RESP_Q047_A,TX_RESP_Q047_B,TX_RESP_Q048_A,TX_RESP_Q048_B,TX_RESP_Q048_C,TX_RESP_Q049_B,TX_RESP_Q049_C,TX_RESP_Q050_A,TX_RESP_Q050_B,TX_RESP_Q054_A,TX_RESP_Q054_C,TX_RESP_Q057_A,TX_RESP_Q057_B,TX_RESP_Q057_C,TX_RESP_Q039_A,TX_RESP_Q039_B,TX_RESP_Q039_C,TX_RESP_Q045_A,TX_RESP_Q045_B,TX_RESP_Q045_C,TX_RESP_Q045_D,TX_RESP_Q051_B,TX_RESP_Q051_D,TX_RESP_Q053_A,TX_RESP_Q053_B,TX_RESP_Q053_D,TX_RESP_Q059_D,TX_RESP_Q060_A,TX_RESP_Q060_B,TX_RESP_Q060_D,TX_RESP_Q005_B,TX_RESP_Q005_C,TX_RESP_Q005_D,TX_RESP_Q005_E,TX_RESP_Q006_A,TX_RESP_Q007_A,TX_RESP_Q007_B,TX_RESP_Q008_C,TX_RESP_Q009_A,TX_RESP_Q009_D,TX_RESP_Q011_E,TX_RESP_Q012_A,TX_RESP_Q012_E,TX_RESP_Q013_B,TX_RESP_Q013_C,TX_RESP_Q013_E
29211590,29,2,1,36,65,62,44,43,44,46,21,63,7,49,24,7,7,30,25,25,32,7,47,16,68,1,0,46,5,58,48,3,20,36,48,8,2,21,2,57,3,58,1,6,51,2,11,11,1,45,2,20,1,0,2,49,4,0,0,7,24,8,41,20,4,3,22,21,46,3,38,0,0,36,0,33,2,0
29032539,29,2,1,52,137,133,107,50,121,74,68,110,29,78,47,22,15,48,56,50,69,8,78,45,142,5,0,74,16,124,82,5,31,77,92,30,3,55,3,109,16,114,2,13,93,4,31,25,4,101,5,38,3,1,4,94,4,3,3,29,38,4,117,16,3,1,47,59,83,4,102,1,0,117,0,23,7,0
29033047,29,2,2,48,95,87,79,62,76,64,35,77,20,55,28,14,3,20,63,28,47,17,52,34,99,0,3,47,17,91,53,5,29,46,74,18,1,37,4,76,10,91,0,8,69,0,37,22,2,67,1,33,1,0,2,72,0,0,0,15,40,3,88,4,1,1,43,57,44,0,84,0,0,78,0,21,0,0
29446740,29,2,1,34,94,83,71,68,82,58,40,82,13,60,29,17,18,27,39,29,51,13,69,18,90,8,3,54,13,92,67,1,19,59,84,8,1,21,2,77,8,80,1,12,67,3,39,10,3,65,1,31,1,0,0,53,8,0,0,16,27,1,75,14,4,1,53,40,53,2,59,0,1,77,1,28,6,1
29236606,29,2,1,33,88,82,84,74,74,48,54,86,11,68,44,14,14,35,44,42,43,11,64,26,98,3,0,60,10,91,74,4,26,60,71,15,3,41,4,78,18,92,0,7,78,6,24,11,2,53,4,43,2,0,2,42,2,0,0,14,22,1,76,15,1,1,41,37,59,5,64,0,0,63,0,27,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,29,2,1,25,52,53,40,54,48,43,17,36,21,39,25,9,8,24,25,24,28,6,39,16,52,6,2,38,7,47,39,2,19,30,46,5,1,10,2,54,1,52,0,4,42,1,22,8,1,40,2,19,0,0,2,41,4,1,0,5,22,8,37,23,2,0,24,20,40,2,36,0,0,33,1,30,2,0
29433410,29,2,1,54,99,88,69,91,86,55,57,93,16,63,43,13,11,41,56,37,55,19,80,30,98,13,2,61,15,100,85,6,25,73,84,18,5,48,5,101,7,92,1,8,91,3,36,17,2,66,0,44,4,1,4,74,4,1,1,13,51,6,54,36,17,3,38,23,75,5,57,0,0,68,0,64,11,0
29456487,29,2,2,36,82,79,78,60,61,53,37,80,11,64,26,17,11,24,43,30,41,12,60,21,93,2,0,37,18,86,69,4,19,56,72,14,1,50,2,56,22,87,0,5,74,1,3,64,1,65,7,23,0,0,1,63,4,1,1,3,26,45,63,21,5,2,21,27,57,3,54,0,0,67,0,30,2,0
29457491,29,2,2,27,47,48,41,34,47,32,22,47,12,34,10,12,3,16,27,25,20,8,38,12,57,0,0,20,12,43,33,3,15,33,47,4,1,19,1,34,15,51,0,9,38,1,2,35,3,42,0,11,2,0,2,34,3,0,2,1,4,49,45,11,1,0,12,26,32,2,46,0,0,53,0,10,1,0


In [ ]:
X_transform_em = minmax(X_embedded)

X_train, X_test, y_train, y_test = train_test_split(X_transform_em, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 63.29 %
Naive Bayes: Acurácia do modelo: 63.29 %
Decision Tree: Acurácia do modelo: 65.82 %
SVM: Acurácia do modelo: 69.62 %
Logistic Regression: Acurácia do modelo: 64.56 %
KNN: Acurácia do modelo: 55.7 %


**Aplicando seleção de atributos (Wrapper)**

In [ ]:
X_wrapper = wrapper_based(X, X_transform, y, 100)

X_wrapper

,ID_UF,ID_MUNICIPIO,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q027_A,TX_RESP_Q028_A,TX_RESP_Q029_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_B,TX_RESP_Q022_A,TX_RESP_Q026_C,TX_RESP_Q032_A,TX_RESP_Q032_B,TX_RESP_Q033_A,TX_RESP_Q033_B,TX_RESP_Q033_C,TX_RESP_Q034_A,TX_RESP_Q034_B,TX_RESP_Q034_C,TX_RESP_Q035_A,TX_RESP_Q035_B,TX_RESP_Q040_A,TX_RESP_Q040_B,TX_RESP_Q040_C,TX_RESP_Q041_A,TX_RESP_Q041_B,TX_RESP_Q041_C,TX_RESP_Q042_A,TX_RESP_Q042_B,TX_RESP_Q046_B,TX_RESP_Q046_C,TX_RESP_Q047_A,TX_RESP_Q047_B,TX_RESP_Q048_A,...,TX_RESP_Q045_A,TX_RESP_Q045_B,TX_RESP_Q045_C,TX_RESP_Q045_D,TX_RESP_Q051_A,TX_RESP_Q051_B,TX_RESP_Q051_D,TX_RESP_Q053_A,TX_RESP_Q053_B,TX_RESP_Q053_C,TX_RESP_Q053_D,TX_RESP_Q059_A,TX_RESP_Q059_D,TX_RESP_Q060_A,TX_RESP_Q060_B,TX_RESP_Q060_D,TX_RESP_Q005_A,TX_RESP_Q005_B,TX_RESP_Q005_C,TX_RESP_Q005_D,TX_RESP_Q005_E,TX_RESP_Q006_A,TX_RESP_Q006_E,TX_RESP_Q007_A,TX_RESP_Q007_B,TX_RESP_Q008_C,TX_RESP_Q008_D,TX_RESP_Q009_A,TX_RESP_Q009_B,TX_RESP_Q009_C,TX_RESP_Q009_D,TX_RESP_Q009_E,TX_RESP_Q011_D,TX_RESP_Q011_E,TX_RESP_Q012_A,TX_RESP_Q012_E,TX_RESP_Q013_B,TX_RESP_Q013_C,TX_RESP_Q013_D,TX_RESP_Q014_A
29211590,29,2900108,2,1,36,36,68,65,62,43,44,23,46,21,7,49,24,7,44,7,30,25,25,32,7,47,16,68,1,0,46,5,17,58,1,48,3,20,36,48,...,45,2,20,1,59,0,2,49,4,15,0,65,0,7,24,8,3,41,20,4,3,22,1,21,46,3,0,38,31,1,0,0,0,0,36,0,33,2,1,0
29032539,29,2900207,2,1,52,95,145,137,133,50,121,22,74,68,29,78,47,22,78,15,48,56,50,69,8,78,45,142,5,0,74,16,52,124,10,82,5,31,77,92,...,101,5,38,3,126,1,4,94,4,43,3,122,3,29,38,4,11,117,16,3,1,47,0,59,83,4,1,102,40,1,1,1,0,0,117,0,23,7,1,6
29033047,29,2900207,2,2,48,49,99,95,87,62,76,24,64,35,20,55,28,14,42,3,20,63,28,47,17,52,34,99,0,3,47,17,36,91,6,53,5,29,46,74,...,67,1,33,1,90,0,2,72,0,29,0,90,0,15,40,3,8,88,4,1,1,43,0,57,44,0,0,84,17,1,0,0,0,0,78,0,21,0,0,3
29446740,29,2900306,2,1,34,65,100,94,83,68,82,14,58,40,13,60,29,17,50,18,27,39,29,51,13,69,18,90,8,3,54,13,28,92,5,67,1,19,59,84,...,65,1,31,1,94,0,0,53,8,40,0,98,0,16,27,1,7,75,14,4,1,53,1,40,53,2,0,59,36,1,0,1,0,1,77,1,28,6,0,1
29236606,29,2900504,2,1,33,64,100,88,82,74,74,27,48,54,11,68,44,14,55,14,35,44,42,43,11,64,26,98,3,0,60,10,30,91,2,74,4,26,60,71,...,53,4,43,2,88,0,2,42,2,58,0,78,0,14,22,1,8,76,15,1,1,41,0,37,59,5,0,64,36,0,0,0,0,0,63,0,27,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29409500,29,2933307,2,1,25,34,61,52,53,54,48,11,43,17,21,39,25,9,34,8,24,25,24,28,6,39,16,52,6,2,38,7,12,47,5,39,2,19,30,46,...,40,2,19,0,53,0,2,41,4,14,1,55,0,5,22,8,0,37,23,2,0,24,0,20,40,2,0,36,24,1,0,0,0,0,33,1,30,2,1,0
29433410,29,2933307,2,1,54,54,111,99,88,91,86,28,55,57,16,63,43,13,70,11,41,56,37,55,19,80,30,98,13,2,61,15,34,100,3,85,6,25,73,84,...,66,0,44,4,102,1,4,74,4,34,1,104,1,13,51,6,1,54,36,17,3,38,0,23,75,5,0,57,53,2,0,0,0,0,68,0,64,11,2,0
29456487,29,2933307,2,2,36,61,91,82,79,60,61,31,53,37,11,64,26,17,50,11,24,43,30,41,12,60,21,93,2,0,37,18,39,86,3,69,4,19,56,72,...,65,7,23,0,89,0,1,63,4,26,1,78,1,3,26,45,5,63,21,5,2,21,0,27,57,3,0,54,38,2,0,0,0,0,67,0,30,2,0,0
29457491,29,2933307,2,2,27,33,55,47,48,34,47,11,32,22,12,34,10,12,29,3,16,27,25,20,8,38,12,57,0,0,20,12,24,43,7,33,3,15,33,47,...,42,0,11,2,49,0,2,34,3,17,0,51,2,1,4,49,3,45,11,1,0,12,0,26,32,2,0,46,13,1,0,0,0,0,53,0,10,1,1,2


In [ ]:
X_transform_w = minmax(X_wrapper)

X_train, X_test, y_train, y_test = train_test_split(X_transform_w, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 64.56 %
Naive Bayes: Acurácia do modelo: 64.56 %
Decision Tree: Acurácia do modelo: 55.7 %
SVM: Acurácia do modelo: 69.62 %
Logistic Regression: Acurácia do modelo: 70.89 %
KNN: Acurácia do modelo: 56.96 %
